<a id='introduction'></a>
# Step 1: Introduction, Core Physics, and Project Goals

This notebook uses the `nrpy` library to construct a complete C-language project for integrating the geodesic paths of **massive particles** (e.g., stars, gas clouds) in a black hole spacetime. The resulting C code is a high-performance N-body engine that evolves a "collisionless gas" and outputs its state at regular time intervals. These "snapshots" of the particle disk can then be used as a physical source for a ray-tracing or radiative transfer code, such as the `photon_geodesic_integrator`.

The core of the project is the numerical solution of the geodesic equation for a massive particle, which is parameterized by proper time `τ`:

$$ \frac{d^2 x^\alpha}{d\tau^2} = -\Gamma^\alpha_{\mu\nu} \frac{dx^\mu}{d\tau} \frac{dx^\nu}{d\tau} $$

To solve this second-order ODE numerically, we decompose it into a system of first-order ODEs by defining the 4-velocity, $u^\alpha \equiv \frac{dx^\alpha}{d\tau}$. This gives us a system of 8 coupled ODEs for the 8-component state vector $y = (x^0, x^1, x^2, x^3, u^0, u^1, u^2, u^3)$, where $x^0 = t$:

1.  **Position ODEs**: $\frac{dx^\alpha}{d\tau} = u^\alpha$
2.  **Velocity ODEs**: $\frac{du^\alpha}{d\tau} = -\Gamma^\alpha_{\mu\nu} u^\mu u^\nu$

A key challenge is that we want to output snapshots of the entire system at a specific **coordinate time `t`**, but each particle's proper time `τ` elapses at a different rate due to relativistic time dilation. This notebook's generated code solves this by using the GNU Scientific Library (GSL) ODE driver, which can integrate the system with respect to coordinate time `t` by implicitly applying the chain rule:

$$ \frac{dy^\alpha}{dt} = \frac{dy^\alpha}{d\tau} \frac{d\tau}{dt} = \frac{1}{u^t} \frac{dy^\alpha}{d\tau} $$

This ensures that all particles can be evolved to the exact same coordinate time for each snapshot.


# Table of Contents

This notebook generates the complete C code for a simulation of massive, non-interacting particles orbiting a black hole.

**Part 1: Introduction & Project Setup**
*   [Step 1: Introduction, Core Physics, and Project Goals](#introduction)
*   [Step 2: Project Initialization and Parameter Definition](#initialize)

**Part 2: The Symbolic Core (The "Recipes")**
*   [Step 3: The Symbolic Core - Foundational Math](#symbolic_core)
    *   [3.a: Metric Tensor Derivatives](#deriv_g4DD)
    *   [3.b: Christoffel Symbol Calculation](#four_connections)
    *   [3.c: Geodesic Equations of Motion](#geodesic_eom)
    *   [3.d: Initial 4-Velocity Conditions](#initial_velocity)
    *   [3.e: Conserved Quantities (E, L, Q)](#conserved_quantities)

**Part 3: Spacetime Definitions & Symbolic Execution**
*   [Step 4: Spacetime Definitions](#spacetime_definition)
*   [Step 5: Symbolic Workflow Execution](#symbolic_execution)

**Part 4: C Code Generation**
*   [Step 6: C Code Generation - Physics "Engines" and "Workers"](#generate_c_engines)
    *   [6.A: Tier 4 - Low-Level Workers, Helpers, and Dispatchers](#tier_4_workers)
    *   [6.B: Tier 3 - Core Subsystems & Engines](#tier_3_engines)
    *   [6.C: Tiers 2 & 1 - Top-Level Orchestrators](#tiers_1_2_orchestrators)

**Part 5: Project Assembly and Compilation**
*   [Step 7: Project Assembly and Compilation](#assemble_project)
    *   [7.a: Registering Core C Data Structures](#register_structs)
    *   [7.b: Final Build Command](#final_build)
"""

<a id='initialize'></a>
# Step 2: Project Initialization and Parameter Definition

This cell sets up the foundational elements for our entire project. It performs three key tasks:

1.  **Import Libraries**: We import necessary modules from standard Python libraries (`os`, `shutil`, `sympy`) and the core components of `nrpy`. The `nrpy` imports provide tools for C function registration, C code generation, parameter handling, and infrastructure management.

2.  **Directory Management**: A clean output directory, `project/mass_integrator/`, is created to store the generated C code, ensuring a fresh build every time the notebook is run.

3.  **Physical and Runtime Parameter Definition**: We define the many parameters that control the simulation using `nrpy`'s `CodeParameter` system. This is the central mechanism for defining a runtime parameter that will be accessible in the generated C code. The `nrpy` build system uses this registry of parameters to automatically construct C data structures, a default parameter file (`mass_integrator.par`), and a robust command-line parser.

In [ ]:
# In V1_5_mass_geodesic.ipynb, Cell ID 33f07e1c (UPDATED)
import os
import shutil
import sympy as sp
import nrpy.c_function as cfc
import nrpy.c_codegen as ccg
import nrpy.params as par
import nrpy.indexedexp as ixp
import nrpy.infrastructures.BHaH.BHaH_defines_h as Bdefines_h
import nrpy.infrastructures.BHaH.Makefile_helpers as Makefile
from nrpy.infrastructures.BHaH import cmdline_input_and_parfiles
import nrpy.helpers.generic as gh
import nrpy.infrastructures.BHaH.CodeParameters as CPs

project_name = "mass_integrator"
project_dir = os.path.join("project", project_name)
shutil.rmtree(project_dir, ignore_errors=True)

par.set_parval_from_str("Infrastructure", "BHaH")

# --- Physical Parameters ---
M_scale = par.CodeParameter("REAL", __name__, "M_scale", 1.0, commondata=True, add_to_parfile=True, add_to_set_CodeParameters_h=True)
a_spin = par.CodeParameter("REAL", __name__, "a_spin", 0.9, commondata=True, add_to_parfile=True, add_to_set_CodeParameters_h=True)

metric_choice = par.CodeParameter(
    "int", __name__, "metric_choice", 0,
    commondata=True, add_to_parfile=True
)

# --- Integration & Termination Parameters ---
t_max_integration = par.CodeParameter("REAL", __name__, "t_max_integration", 2000.0, commondata=True, add_to_parfile=True)
flatness_threshold = par.CodeParameter("REAL", __name__, "flatness_threshold", 1e-2, commondata=True, add_to_parfile=True)
r_escape = par.CodeParameter("REAL", __name__, "r_escape", 1500.0, commondata=True, add_to_parfile=True)
ut_max = par.CodeParameter("REAL", __name__, "ut_max", 1e3, commondata=True, add_to_parfile=True)

# --- Debugging & Validation Parameters ---
perform_conservation_check = par.CodeParameter("bool", __name__, "perform_conservation_check", True, commondata=True, add_to_parfile=True)
run_in_debug_mode = par.CodeParameter("bool", __name__, "run_in_debug_mode", True, commondata=True, add_to_parfile=True)

# --- Disk Parameters ---
disk_lambda_rest_at_r_min = par.CodeParameter("REAL", __name__, "disk_lambda_rest_at_r_min", 656.3, commondata=True, add_to_parfile=True)
disk_num_r= par.CodeParameter("int", __name__, "disk_num_r", 100, commondata=True, add_to_parfile=True)
disk_num_phi= par.CodeParameter("int", __name__, "disk_num_phi", 200, commondata=True, add_to_parfile=True)
disk_r_min = par.CodeParameter("REAL", __name__, "disk_r_min", 6.0, commondata=True, add_to_parfile=True)
disk_r_max = par.CodeParameter("REAL", __name__, "disk_r_max", 25.0, commondata=True, add_to_parfile=True)
snapshot_every_t = par.CodeParameter("REAL", __name__, "snapshot_every_t", 10.0, commondata=True, add_to_parfile=True)
t_final = par.CodeParameter("REAL", __name__, "t_final", 2000.0, commondata=True, add_to_parfile=True)
output_folder = par.CodeParameter("char[100]", __name__, "output_folder", "output", commondata=True, add_to_parfile=True)

# --- : Barred Flocculent Spiral Galaxy Shape Parameters ---
print("-> Registering CodeParameters for barred flocculent spiral galaxy geometry...")
bar_length = par.CodeParameter("REAL", __name__, "bar_length", 5.0, commondata=True, add_to_parfile=True)
bar_aspect_ratio = par.CodeParameter("REAL", __name__, "bar_aspect_ratio", 0.25, commondata=True, add_to_parfile=True) # width / length
bulge_radius = par.CodeParameter("REAL", __name__, "bulge_radius", 1.5, commondata=True, add_to_parfile=True)
arm_particle_density = par.CodeParameter("REAL", __name__, "arm_particle_density", 0.3, commondata=True, add_to_parfile=True) # Base probability of a particle being in an arm
arm_clumpiness_factor = par.CodeParameter("REAL", __name__, "arm_clumpiness_factor", 8.0, commondata=True, add_to_parfile=True) # How many clumps per arm rotation
arm_clump_size = par.CodeParameter("REAL", __name__, "arm_clump_size", 0.5, commondata=True, add_to_parfile=True) # How tight the clumps are
bar_density_factor = par.CodeParameter("REAL", __name__, "bar_density_factor", 2.0, commondata=True, add_to_parfile=True) # Bar is 2x denser than arms
bulge_density_factor = par.CodeParameter("REAL", __name__, "bulge_density_factor", 3.0, commondata=True, add_to_parfile=True) # Bulge is 3x denser

# --- : Spiral Galaxy Shape Parameters ---
print("-> Registering CodeParameters for spiral galaxy geometry...")
spiral_galaxy_num_arms = par.CodeParameter(
    "int", __name__, "spiral_galaxy_num_arms", 2,
    commondata=True, add_to_parfile=True
)
spiral_galaxy_arm_tightness = par.CodeParameter(
    "REAL", __name__, "spiral_galaxy_arm_tightness", 0.2,
    commondata=True, add_to_parfile=True
)

# --- : Initial Conditions Type Selector ---
print("-> Registering CodeParameter for selecting initial conditions type...")
initial_conditions_type = par.CodeParameter("char[100]", __name__, "initial_conditions_type", "KeplerianDisk",commondata=True, add_to_parfile=True)

<a id='symbolic_core'></a>
# Step 3: The Symbolic Core - Foundational Math

This section defines the pure mathematical logic of our problem using Python's `sympy` library. Each function in this section is a "blueprint" or "recipe" for a physical calculation. These functions take symbolic `sympy` objects as input (like a metric tensor) and return new symbolic expressions as output (like the Christoffel symbols). They have no knowledge of C code; they are concerned only with mathematics and will be called later in Step 5 to generate the explicit formulas for our C code engines.


<a id='deriv_g4DD'></a>
### 3.a: Metric Tensor Derivatives

The first step in calculating the Christoffel symbols is to compute the partial derivatives of the metric tensor, $g_{\mu\nu}$. This function, `derivative_g4DD`, takes the symbolic 4x4 metric tensor `g4DD` and a list of the four coordinate symbols `xx` as input.

The function iterates through all components to symbolically calculate the partial derivative of each metric component with respect to each coordinate. The resulting quantity, which we can denote using comma notation as $g_{\mu\nu,\alpha}$, is defined as:

$$ g_{\mu\nu,\alpha} \equiv \frac{\partial g_{\mu\nu}}{\partial x^{\alpha}} $$

The nested `for` loops in the code directly correspond to the spacetime indices `μ, ν, α` in the physics equation. `sympy`'s built-in `sp.diff()` function is used to perform the symbolic differentiation, and the final result is returned as a rank-3 symbolic tensor.

### `nrpy` Functions Used in this Cell:

*   **`nrpy.indexedexp.zerorank3(dimension)`**:
    *   **Source File**: `nrpy/indexedexp.py`
    *   **Description**: This function creates a symbolic rank-3 tensor (a Python list of lists of lists) of a specified dimension, with all elements initialized to the `sympy` integer 0. It is used here to create a container for the derivative results.


In [ ]:
def derivative_g4DD(g4DD, xx):
    """Computes the symbolic first derivatives of the metric tensor."""
    g4DD_dD = ixp.zerorank3(dimension=4)
    for nu in range(4):
        for mu in range(4):
            for alpha in range(4):
                g4DD_dD[nu][mu][alpha] = sp.diff(g4DD[nu][mu], xx[alpha])
    return g4DD_dD

<a id='four_connections'></a>
### 3.b: Christoffel Symbol Calculation

This function implements the core formula for the Christoffel symbols of the second kind, $\Gamma^{\delta}_{\mu\nu}$. It takes the symbolic metric tensor `g4DD` ($g_{\mu\nu}$) and its derivatives `g4DD_dD` ($g_{\mu\nu,\alpha}$) as input. The calculation requires the inverse metric, $g^{\mu\nu}$, which is computed using another `nrpy` helper function.

The function then applies the well-known formula for the Christoffel symbols. Using the comma notation for partial derivatives, the formula is:

$$ \Gamma^{\delta}_{\mu\nu} = \frac{1}{2} g^{\delta\alpha} \left( g_{\nu\alpha,\mu} + g_{\mu\alpha,\nu} - g_{\mu\nu,\alpha} \right) $$

The Python `for` loops iterate over the spacetime indices `δ, μ, ν, α` to construct each component of the Christoffel symbol tensor. After the summation is complete, the `sp.trigsimp()` function is used to simplify the resulting expression. This trigonometric simplification is highly effective and much faster than a general `sp.simplify()` for the Kerr-Schild metric, which contains trigonometric functions of the coordinates.

### `nrpy` Functions Used in this Cell:

*   **`nrpy.indexedexp.zerorank3(dimension)`**: Previously introduced. Used to initialize the Christoffel symbol tensor.
*   **`nrpy.indexedexp.symm_matrix_inverter4x4(g4DD)`**:
    *   **Source File**: `nrpy/indexedexp.py`
    *   **Description**: This function takes a symbolic 4x4 symmetric matrix and analytically computes its inverse. It is highly optimized for this specific task, returning both the inverse matrix ($g^{\mu\nu}$) and its determinant.

In [ ]:
def four_connections(g4DD, g4DD_dD):
    """
    Computes and simplifies Christoffel symbols from the metric and its derivatives.
    
    This version uses sp.trigsimp() which is highly effective and much faster
    than sp.simplify() for the Kerr-Schild metric.
    """
    Gamma4UDD = ixp.zerorank3(dimension=4)
    g4UU, _ = ixp.symm_matrix_inverter4x4(g4DD)
    
    for mu in range(4):
        for nu in range(4):
            for delta in range(4):
                # Calculate the Christoffel symbol component using the standard formula
                for alpha in range(4):
                    Gamma4UDD[delta][mu][nu] += sp.Rational(1, 2) * g4UU[delta][alpha] * \
                        (g4DD_dD[nu][alpha][mu] + g4DD_dD[mu][alpha][nu] - g4DD_dD[mu][nu][alpha])
                
                # Use sp.trigsimp() to simplify the resulting expression.
                # This is the key to speeding up the symbolic calculation.
                Gamma4UDD[delta][mu][nu] = sp.trigsimp(Gamma4UDD[delta][mu][nu])

    return Gamma4UDD

<a id='geodesic_eom'></a>
### 3.c: Geodesic Equations of Motion

The following two functions define the right-hand sides (RHS) for our system of 8 first-order ODEs, which describe the evolution of a massive particle's position and 4-velocity.

**Velocity ODE:**
The core of geodesic motion is described by the evolution of the 4-velocity, $u^\alpha$:
$$ \frac{du^{\alpha}}{d\tau} = -\Gamma^{\alpha}_{\mu\nu} u^{\mu} u^{\nu} $$
The function `geodesic_vel_rhs_massive` constructs the symbolic expression for this RHS. It performs the Einstein summation $-\Gamma^{\alpha}_{\mu\nu} u^{\mu} u^{\nu}$ using a symmetry optimization. Since the Christoffel symbols are symmetric in their lower two indices ($\Gamma^{\alpha}_{\mu\nu} = \Gamma^{\alpha}_{\nu\mu}$), we can write the sum as:
$$ \sum_{\mu=0}^3 \sum_{\nu=0}^3 \Gamma^{\alpha}_{\mu\nu} u^{\mu} u^{\nu} = \sum_{\mu=0}^3 \Gamma^{\alpha}_{\mu\mu} (u^{\mu})^2 + 2 \sum_{\mu=0}^3 \sum_{\nu=\mu+1}^3 \Gamma^{\alpha}_{\mu\nu} u^{\mu} u^{\nu} $$
This reduces the number of terms to compute from 16 to 10 for each component $\alpha$, generating more efficient C code.

In [ ]:
def geodesic_vel_rhs_massive():
    """
    Symbolic RHS for massive particle velocity ODE: du^a/dτ = -Γ^a_μν u^μ u^ν.
    u is the 4-velocity, y[4]...y[7].
    """
    Gamma4UDD = ixp.declarerank3("conn->Gamma4UDD",dimension= 4,sym="sym12")
    ut,ux,uy,uz = sp.symbols("y[4] y[5] y[6] y[7]", Real=True)
    uU = [ut,ux,uy,uz]
    geodesic_rhs = ixp.zerorank1(dimension=4)
    for alpha in range(4):
        for mu in range(4):
            geodesic_rhs[alpha] += Gamma4UDD[alpha][mu][mu] * uU[mu] * uU[mu]
            for nu in range(mu + 1, 4):
                geodesic_rhs[alpha] += 2 * Gamma4UDD[alpha][mu][nu] * uU[mu] * uU[nu]
        geodesic_rhs[alpha] = -geodesic_rhs[alpha]
    return geodesic_rhs


**Position ODE:**
The evolution of the position coordinates, $x^\alpha$, is simply given by the definition of the 4-velocity:
$$ \frac{dx^{\alpha}}{d\tau} = u^{\alpha} $$
The function `geodesic_pos_rhs_massive` implements this straightforward relationship, returning the symbolic 4-velocity components that will become the RHS for the position part of the ODE system.

In [ ]:
def geodesic_pos_rhs_massive():
    """
    Symbolic RHS for position ODE: dx^a/dτ = u^a.
    u is the 4-velocity, y[4]...y[7].
    """
    ut,ux,uy,uz = sp.symbols("y[4] y[5] y[6] y[7]", Real=True)
    uU = [ut,ux,uy,uz]
    return uU

<a id='initial_velocity'></a>
### 3.d: Initial 4-Velocity Conditions

To set the initial conditions for a particle, we must ensure its 4-velocity is physically valid. This involves two key constraints.

**Timelike Normalization Condition:**
The 4-velocity of a massive particle must be timelike, meaning its magnitude is normalized to -1:
$$ g_{\mu\nu} u^\mu u^\nu = -1 $$
This condition arises from the definition of proper time, $d\tau^2 = -ds^2 = -g_{\mu\nu}dx^\mu dx^\nu$. Dividing by $d\tau^2$ gives $1 = -g_{\mu\nu}\frac{dx^\mu}{d\tau}\frac{dx^\nu}{d\tau} = -g_{\mu\nu}u^\mu u^\nu$.

The function `ut_massive()` symbolically solves this quadratic equation for the time component of the 4-velocity, $u^t = u^0$. This allows us to compute $u^t$ if we know the spatial components ($u^i$).

To derive the solution, we expand the summation:
$$ g_{00}(u^0)^2 + g_{0i}u^0 u^i + g_{i0}u^i u^0 + g_{ij}u^i u^j = -1 $$
Using the symmetry of the metric ($g_{\mu\nu} = g_{\nu\mu}$), this becomes a standard quadratic equation of the form $A(u^0)^2 + B(u^0) + C = 0$:
$$ \underbrace{g_{00}}_{A}(u^0)^2 + \underbrace{2g_{0i}u^i}_{B}u^0 + \underbrace{(g_{ij}u^i u^j + 1)}_{C} = 0 $$
The function then applies the quadratic formula, $u^0 = \frac{-B \pm \sqrt{B^2 - 4AC}}{2A}$, to solve for $u^0$. The negative sign in the numerator is chosen to ensure the particle moves forward in coordinate time for typical spacetimes outside the event horizon.


In [ ]:
def ut_massive():
    """
    Symbolically derives u^t for a MASSIVE particle from the 4-velocity.
    The derivation comes from solving the timelike normalization condition g_μν u^μ u^ν = -1.
    """
    # The symbolic recipe will use the standard variable names u0, u1, etc.
    # The C-generating function will map y[4], y[5], etc. to these.
    u0,u1,u2,u3 = sp.symbols("u0 u1 u2 u3", Real=True)
    uU=[u0,u1,u2,u3]
    
    # The recipe uses the standard name "metric" for the struct.
    g4DD = ixp.declarerank2("metric->g", sym="sym01", dimension=4)

    # This is the quadratic equation for u^0, derived from g_μν u^μ u^ν = -1
    # g_00(u^0)^2 + 2g_0i u^0 u^i + g_ij u^i u^j = -1
    # We solve for u^0.
    sum_g0i_ui = sp.sympify(0)
    for i in range(1,4):
        sum_g0i_ui += g4DD[0][i]*uU[i]
        
    sum_gij_ui_uj = sp.sympify(0)
    for i in range(1,4):
        sum_gij_ui_uj += g4DD[i][i]*uU[i]*uU[i]
        for j in range(i+1,4):
            sum_gij_ui_uj += 2*g4DD[i][j]*uU[i]*uU[j]
            
    # The discriminant of the quadratic formula for u^0
    # CORRECTED: This now includes the "+1" term from g_μν u^μ u^ν = -1
    discriminant = sum_g0i_ui**2 - g4DD[0][0]*(sum_gij_ui_uj + 1)
    
    # We choose the positive root for a forward-in-time particle outside the horizon.
    # Note: Your choice of the minus sign was correct for the final expression.
    answer = (-sum_g0i_ui - sp.sqrt(discriminant)) / g4DD[0][0]
    return answer

**Stable Circular Orbits:**
For setting up particles in a stable accretion disk, we often start with circular, equatorial orbits. The function `symbolic_ut_uphi_from_r_stable()` provides a highly accurate and numerically stable method to compute the initial $u^t$ and angular velocity $u^\phi$ for a particle at a given radius `r`. It uses a specialized three-step method that avoids the catastrophic cancellation that can occur in the general formula when orbits are very close to the black hole.

The method is as follows:
1.  **Calculate Angular Velocity $\Omega$**: First, it computes the coordinate angular velocity $\Omega = d\phi/dt$ for a stable prograde circular orbit in the equatorial plane of a Kerr black hole:
    $$ \Omega = \frac{\sqrt{M}}{r^{3/2} + a\sqrt{M}} $$
2.  **Solve for $u^t$**: It then uses the 4-velocity normalization condition, $g_{\mu\nu}u^\mu u^\nu = -1$. For a circular orbit, $u^r = u^\theta = 0$, and we have the relation $u^\phi = (d\phi/dt) (dt/d\tau) = \Omega u^t$. Substituting this into the normalization condition gives:
    $$ g_{tt}(u^t)^2 + 2g_{t\phi}u^t u^\phi + g_{\phi\phi}(u^\phi)^2 = -1 $$
    $$ (u^t)^2 (g_{tt} + 2g_{t\phi}\Omega + g_{\phi\phi}\Omega^2) = -1 $$
    This is solved for $u^t$:
    $$ u^t = \frac{1}{\sqrt{-(g_{tt} + 2g_{t\phi}\Omega + g_{\phi\phi}\Omega^2)}} $$
3.  **Calculate $u^\phi$**: Finally, it computes $u^\phi$ from the previously found quantities:
    $$ u^\phi = \Omega u^t $$
This three-step process is more numerically stable than directly solving a more complex single equation, especially for orbits near the Innermost Stable Circular Orbit (ISCO).


In [ ]:
def symbolic_ut_uphi_from_r_stable():
    """
    Symbolically derives u^t and u^phi for a circular, equatorial orbit
    using a NUMERICALLY STABLE three-step method.

    This definitive version constructs all expressions using the final C-level
    parameter names directly, avoiding the need for .subs() and adhering to
    the nrpy design pattern.
    
    Returns a list containing the symbolic expressions for [u^t, u^phi].
    """
    # --- Step 0: Define symbolic variables with final C names ---
    # These symbols directly correspond to the variables that will be
    # available in the C function that uses this recipe.
    r, M, a = sp.symbols("r_initial M_scale a_spin", real=True)
    
    # --- Step 1: Calculate the stable angular velocity Omega = d(phi)/dt ---
    Omega = sp.sqrt(M) / (r**sp.Rational(3, 2) + a * sp.sqrt(M))
    
    # --- Step 2: Define the required metric components using the same symbols ---
    # These are the Kerr metric components in Boyer-Lindquist coordinates,
    # specialized to the equatorial plane (theta = pi/2).
    g_tt = -(1 - 2*M/r)
    g_tphi = -2*a*M/r
    g_phiphi = r**2 + a**2 + (2*M*a**2)/r

    # --- Step 3: Solve for u^t using the 4-velocity normalization condition ---
    # (u^t)^2 = -1 / (g_tt + 2*g_tphi*Omega + g_phiphi*Omega^2)
    ut_squared_inv_denom = g_tt + 2*g_tphi*Omega + g_phiphi*Omega**2
    ut_squared = -1 / ut_squared_inv_denom
    ut = sp.sqrt(ut_squared)
    
    # --- Step 4: Calculate u^phi ---
    # u^phi = Omega * u^t
    uphi = Omega * ut
    
    return [ut, uphi]

<a id='conserved_quantities'></a>
### 3.e: Conserved Quantities (E, L, Q)

For particles moving in a Kerr spacetime, certain physical quantities are conserved along their geodesic paths due to the spacetime's symmetries. These are invaluable for validating the numerical accuracy of our integrator.

**Energy at Infinity (E):**
The energy is conserved due to the time-independence (stationarity) of the Kerr spacetime. It is defined as the projection of the particle's 4-momentum $p^\mu$ onto the time-like Killing vector, which corresponds to the covariant time component of the 4-momentum:
$$ E = -p_t = -g_{t\mu}p^\mu $$
For massive particles, we can set the particle's mass to 1, so the 4-momentum $p^\mu$ is equivalent to the 4-velocity $u^\mu$. The function `symbolic_energy()` implements this formula.


In [ ]:
def symbolic_energy():
    """
    Computes the symbolic expression for conserved energy E = -p_t.
    E = -g_{t,mu} p^mu
    """
    # Define the 4-momentum components using the y[4]...y[7] convention
    pt, px, py, pz = sp.symbols("y[4] y[5] y[6] y[7]", real=True)
    pU = [pt, px, py, pz]
    
    # Define an abstract metric tensor to be filled by a C struct at runtime
    g4DD = ixp.declarerank2("metric->g", sym="sym01", dimension=4)
    
    # Calculate p_t = g_{t,mu} p^mu
    p_t = sp.sympify(0)
    for mu in range(4):
        p_t += g4DD[0][mu] * pU[mu]
        
    return -p_t


**Angular Momentum (L):**
The angular momentum vector $\vec{L}$ is defined in the usual way from classical mechanics, $\vec{L} = \vec{x} \times \vec{p}$, but using the covariant components of the 4-momentum:
$$ L_i = \epsilon_{ijk} x^j p_k $$
where $\epsilon_{ijk}$ is the Levi-Civita symbol, and the covariant momentum components $p_k$ are found by lowering the index of the contravariant 4-momentum $p^\mu$:
$$ p_k = g_{k\mu} p^\mu $$
The function `symbolic_L_components_cart()` first calculates the covariant spatial components of the momentum ($p_x, p_y, p_z$) and then uses them to compute the three Cartesian components of the angular momentum vector ($L_x, L_y, L_z$). For an axisymmetric spacetime like Kerr, the component parallel to the axis of symmetry, $L_z$, is a conserved quantity.


In [ ]:
def symbolic_L_components_cart():
    """
    Computes the symbolic expressions for the three components of angular momentum,
    correctly accounting for the symmetry of the metric tensor.
    """
    # Define coordinate and 4-momentum components
    t, x, y, z = sp.symbols("y[0] y[1] y[2] y[3]", real=True)
    pt, px, py, pz = sp.symbols("y[4] y[5] y[6] y[7]", real=True)
    pU = [pt, px, py, pz]
    
    # Define an abstract metric tensor
    g4DD = ixp.declarerank2("metric->g", sym="sym01", dimension=4)
    
    # --- THIS IS THE CORE FIX ---
    # Calculate covariant momentum components p_k = g_{k,mu} p^mu,
    # correctly exploiting the metric's symmetry g_mu,nu = g_nu,mu.
    p_down = ixp.zerorank1(dimension=4)
    for k in range(1, 4): # We only need p_x, p_y, p_z for L_i
        # Sum over mu
        for mu in range(4):
            # Use g4DD[k][mu] if k <= mu, otherwise use g4DD[mu][k]
            if k <= mu:
                p_down[k] += g4DD[k][mu] * pU[mu]
            else: # k > mu
                p_down[k] += g4DD[mu][k] * pU[mu]
            
    p_x, p_y, p_z = p_down[1], p_down[2], p_down[3]

    # Calculate angular momentum components 
    L_x = y*p_z - z*p_y
    L_y = z*p_x - x*p_z
    L_z = x*p_y - y*p_x
    
    return [L_x, L_y, L_z]

**Carter Constant (Q):**
The Carter Constant is a third, more complex conserved quantity related to a hidden symmetry of the Kerr metric. In the Schwarzschild case (spin a=0), it simplifies to the square of the total angular momentum, $L^2 = L_x^2 + L_y^2 + L_z^2$. For the general Kerr case, it is given by:
$$ Q = p_\theta^2 + \cos^2\theta \left( a^2(m^2 - E^2) + \frac{L_z^2}{\sin^2\theta} \right) $$
For massive particles where we set the mass $m=1$, this becomes:
$$ Q = p_\theta^2 + \cos^2\theta \left( a^2(1 - E^2) + \frac{L_z^2}{\sin^2\theta} \right) $$
The function `symbolic_carter_constant_Q()` implements this formula. A key challenge is that the formula is expressed in spherical coordinates, while our simulation runs in Cartesian coordinates. The function robustly computes all terms directly from the Cartesian position and momentum components. For example, $p_\theta^2$ is derived from the transformation of momentum components, and trigonometric terms are replaced by their Cartesian equivalents:
$$ \cos^2\theta = \frac{z^2}{r^2}, \quad \sin^2\theta = \frac{x^2+y^2}{r^2} = \frac{\rho^2}{r^2} $$
This avoids coordinate transformations and handles the axial singularity (where $\rho^2 = x^2+y^2 \to 0$, leading to division by zero in the $L_z^2/\sin^2\theta$ term) by using a `Piecewise` function. For motion on the z-axis, $L_z=0$ and $p_\theta=0$, so $Q$ is correctly set to 0, preventing numerical errors.


In [ ]:
def symbolic_carter_constant_Q():
    """
    Computes the symbolic expression for the Carter Constant Q using a
    verified formula, robustly handling the axial singularity.
    """
    # Define all necessary symbolic variables
    t, x, y, z = sp.symbols("y[0] y[1] y[2] y[3]", real=True)
    pt, px, py, pz = sp.symbols("y[4] y[5] y[6] y[7]", real=True)
    pU = [pt, px, py, pz]
    a = sp.Symbol("a_spin", real=True)
    g4DD = ixp.declarerank2("metric->g", sym="sym01", dimension=4)

    # --- Step 1: Compute intermediate quantities E, Lz, and p_i ---
    E = symbolic_energy()
    _, _, Lz = symbolic_L_components_cart()
    
    p_down = ixp.zerorank1(dimension=4)
    for k in range(1, 4):
        for mu in range(4):
            if k <= mu: p_down[k] += g4DD[k][mu] * pU[mu]
            else: p_down[k] += g4DD[mu][k] * pU[mu]
    p_x, p_y, p_z = p_down[1], p_down[2], p_down[3]

    # --- Step 2: Compute geometric terms ---
    r_sq = x**2 + y**2 + z**2
    rho_sq = x**2 + y**2
    
    # --- Step 3: Compute p_theta^2 directly in Cartesian components ---
    # This avoids square roots and potential complex number issues in sympy.
    # p_theta^2 = r^2 * p_z^2 + cot^2(theta) * (x*p_x + y*p_y)^2 - 2*r*p_z*cot(theta)*(x*p_x+y*p_y)
    # where cot(theta) = z / rho
    
    # This term is (x*p_x + y*p_y)
    xpx_plus_ypy = x*p_x + y*p_y
    
    # This is p_theta^2, constructed to avoid dividing by rho before squaring.
    # It is equivalent to (z*xpx_plus_ypy/rho - rho*p_z)^2
    p_theta_sq = (z**2 * xpx_plus_ypy**2 / rho_sq) - (2 * z * p_z * xpx_plus_ypy) + (rho_sq * p_z**2)

    # --- Step 4: Assemble the final formula for Q ---
    # Q = p_theta^2 + cos^2(theta) * (-a^2*E^2 + L_z^2/sin^2(theta))
    # where cos^2(theta) = z^2/r^2 and sin^2(theta) = rho^2/r^2
    
    # This is the second term in the Q formula
    second_term = (z**2 / r_sq) * (-a**2 * E**2 + Lz**2 * (r_sq / rho_sq))
    
    Q_formula = p_theta_sq + second_term
    
    # --- Step 5: Handle the axial singularity ---
    # For motion on the z-axis (rho_sq -> 0), Lz=0 and p_theta=0, so Q=0.
    Q_final = sp.Piecewise(
        (0, rho_sq < 1e-12),
        (Q_formula, True)
    )
    
    return Q_final

print("Final symbolic recipes for conserved quantities defined (Carter Constant re-derived).")

<a id='spacetime_definition'></a>
# Step 4: Spacetime Definitions

This section defines the specific spacetime geometries in which the particles will move. We provide symbolic recipes for two key analytic spacetimes in Cartesian coordinates.

**Kerr-Schild Metric:**
The function `define_kerr_metric_Cartesian_Kerr_Schild()` defines the Kerr metric in **Cartesian Kerr-Schild coordinates**. This coordinate system is highly advantageous for numerical work because it is "regular" everywhere, including at the event horizon. This means the metric components do not blow up, allowing the integrator to smoothly trace an orbit across the horizon without encountering coordinate singularities.

The metric is constructed using the formula:
$$ g_{\mu\nu} = \eta_{\mu\nu} + 2H l_\mu l_\nu $$
where:
*   $\eta_{\mu\nu}$ is the flat Minkowski metric, `diag(-1, 1, 1, 1)`.
*   $l_\mu$ is a null vector field (i.e., $g^{\mu\nu}l_\mu l_\nu = 0$). Its components are:
    $$ l_t = 1, \quad l_x = \frac{rx + ay}{r^2 + a^2}, \quad l_y = \frac{ry - ax}{r^2 + a^2}, \quad l_z = \frac{z}{r} $$
*   $H$ is a scalar function given by:
    $$ H = \frac{Mr^3}{r^4 + a^2 z^2} $$
A key feature is that if the spin parameter `a_spin` is set to zero, the null vector simplifies and the metric automatically and exactly reduces to the Schwarzschild metric, allowing a single set of C functions to handle both rotating and non-rotating black holes.


In [ ]:
def define_kerr_metric_Cartesian_Kerr_Schild():
    """
    Defines the Kerr metric tensor in Cartesian Kerr-Schild coordinates.

    This function is the new, unified source for both Kerr (a != 0) and
    Schwarzschild (a = 0) spacetimes. The coordinates are (t, x, y, z).
    
    Returns:
        A tuple (g4DD, xx), where g4DD is the symbolic 4x4 metric tensor
        and xx is the list of symbolic coordinate variables.
    """
    # Define the symbolic coordinates using the 'y[i]' convention for the integrator
    t, x, y, z = sp.symbols("y[0] y[1] y[2] y[3]", real=True)
    xx = [t, x, y, z]

    # Access the symbolic versions of the mass and spin parameters
    M = M_scale.symbol
    a = a_spin.symbol

    # Define intermediate quantities
    r2 = x**2 + y**2 + z**2
    r = sp.sqrt(r2)
    
    # Define the Kerr-Schild null vector l_μ
    l_down = ixp.zerorank1(dimension=4)
    l_down[0] = 1
    l_down[1] = (r*x + a*y) / (r2 + a**2)
    l_down[2] = (r*y - a*x) / (r2 + a**2)
    l_down[3] = z/r

    # Define the scalar function H
    H = (M * r**3) / (r**4 + a**2 * z**2)

    # The Kerr-Schild metric is g_μν = η_μν + 2H * l_μ * l_ν
    # where η_μν is the Minkowski metric diag(-1, 1, 1, 1)
    g4DD = ixp.zerorank2(dimension=4)
    for mu in range(4):
        for nu in range(4):
            eta_mu_nu = 0
            if mu == nu:
                eta_mu_nu = 1
            if mu == 0 and nu == 0:
                eta_mu_nu = -1
            
            g4DD[mu][nu] = eta_mu_nu + 2 * H * l_down[mu] * l_down[nu]
            
    return g4DD, xx

**Standard Schwarzschild Metric:**
The function `define_schwarzschild_metric_cartesian()` provides an alternative definition for the non-rotating black hole spacetime using the standard textbook formula in Cartesian coordinates. This serves as an important cross-validation case for the more general Kerr-Schild implementation.

The metric components are given by:
*   Time-time component:
    $$ g_{tt} = -\left(1 - \frac{2M}{r}\right) $$
*   Time-space components:
    $$ g_{ti} = 0 $$
*   Space-space components:
    $$ g_{ij} = \delta_{ij} + \frac{2M}{r} \frac{x_i x_j}{r^2} $$
where $r = \sqrt{x^2 + y^2 + z^2}$, $\delta_{ij}$ is the Kronecker delta, and $x_i = (x, y, z)$.

In [ ]:
def define_schwarzschild_metric_cartesian():
    """
    Defines the Schwarzschild metric tensor directly in Cartesian coordinates.
    
    This version uses the standard textbook formula and ensures all components
    are sympy objects to prevent C-generation errors.
    
    Returns:
        A tuple (g4DD, xx), where g4DD is the symbolic 4x4 metric tensor
        and xx is the list of symbolic coordinate variables.
    """
    # Define Cartesian coordinates
    t, x, y, z = sp.symbols("y[0] y[1] y[2] y[3]", real=True)
    xx = [t, x, y, z]

    # Access the symbolic mass parameter
    M = M_scale.symbol

    # Define r in terms of Cartesian coordinates
    r = sp.sqrt(x**2 + y**2 + z**2)

    # Define the Cartesian Schwarzschild metric components directly
    g4DD = ixp.zerorank2(dimension=4)
    
    # g_tt
    g4DD[0][0] = -(1 - 2*M/r)
    
    # Spatial components g_ij = δ_ij + (2M/r) * (x_i * x_j / r^2)
    x_i = [x, y, z]
    for i in range(3):
        for j in range(3):
            # --- CORRECTED: Use sp.sympify() for the kronecker delta ---
            delta_ij = sp.sympify(0)
            if i == j:
                delta_ij = sp.sympify(1)
            
            # The indices for g4DD are off by 1 from the spatial indices
            g4DD[i+1][j+1] = delta_ij + (2*M/r) * (x_i[i] * x_i[j] / (r**2))

    # --- CORRECTED: Ensure time-space components are sympy objects ---
    g4DD[0][1] = g4DD[1][0] = sp.sympify(0)
    g4DD[0][2] = g4DD[2][0] = sp.sympify(0)
    g4DD[0][3] = g4DD[3][0] = sp.sympify(0)
            
    return g4DD, xx

<a id='symbolic_execution'></a>
# Step 5: Symbolic Workflow Execution

This cell acts as the central hub for the symbolic portion of our project. In the preceding cells, we *defined* a series of Python functions that perform individual mathematical tasks. Here, we *execute* those functions in the correct sequence to generate all the final symbolic expressions that will serve as "recipes" for our C code generators.

This "symbolic-first" approach is a core `nrpy` principle and offers significant advantages:
1.  **Efficiency**: The complex symbolic calculations, such as inverting the metric tensor and deriving the Christoffel symbols, are performed **only once** when this notebook is run. The results are stored in global Python variables, preventing redundant and time-consuming recalculations. This is especially important for the Kerr metric, whose Christoffel symbols can take several minutes to compute.
2.  **Modularity**: This workflow creates a clean separation between the *specific solution* for a metric (e.g., the explicit formulas for the Kerr-Schild Christoffels) and the *generic form* of the equations of motion (which are valid for any metric).

This cell produces two key sets of symbolic expressions that are stored in global variables for later use:
*   **`Gamma4UDD_kerr`**: The explicit symbolic formulas for the Christoffel symbols of the unified Kerr-Schild metric.
*   **`all_rhs_expressions_massive`**: A Python list containing the 8 symbolic expressions for the right-hand-sides of our generic ODE system. To achieve this generality, we create a symbolic **placeholder** for the Christoffel symbols using `ixp.declarerank3("conn->Gamma4UDD", ...)`. This placeholder is passed to `geodesic_vel_rhs_massive()` to construct the geodesic equation in its abstract form. This elegant technique embeds the final C variable name (`conn->Gamma4UDD...`) directly into the symbolic expression, which dramatically simplifies the C code generation step for the `calculate_ode_rhs_massive()` engine.

### `nrpy` Functions Used in this Cell:

*   **`nrpy.indexedexp.declarerank3(name, dimension, sym)`**:
    *   **Source File**: `nrpy/indexedexp.py`
    *   **Description**: Creates a symbolic tensor of a given rank and dimension. The `name` argument is crucial here; it becomes the base name for the symbolic variables (e.g., `conn->Gamma4UDD011`). This allows us to build generic equations that already contain the C variable names we will use later. The `sym` argument specifies symmetries (e.g., `sym="sym12"` indicates symmetry in the last two indices), which `nrpy` uses to reduce the number of unique symbolic variables created.


In [ ]:
# In V1_5_mass_geodesic.ipynb, Cell ID 5fbfe0b5 (UPDATED)

# --- 1. Define the Kerr-Schild metric and get its derivatives ---
print(" -> Computing Kerr-Schild metric and Christoffel symbols...")
g4DD_kerr, xx_kerr = define_kerr_metric_Cartesian_Kerr_Schild()
g4DD_dD_kerr = derivative_g4DD(g4DD_kerr, xx_kerr)
Gamma4UDD_kerr = four_connections(g4DD_kerr, g4DD_dD_kerr)
print("    ... Done.")

# --- 2. Define the Standard Schwarzschild metric in Cartesian and get its derivatives ---
print(" -> Computing Standard Schwarzschild (Cartesian) metric and Christoffel symbols...")
g4DD_schw_cart, xx_schw_cart = define_schwarzschild_metric_cartesian()
g4DD_dD_schw_cart = derivative_g4DD(g4DD_schw_cart, xx_schw_cart)
Gamma4UDD_schw_cart = four_connections(g4DD_schw_cart, g4DD_dD_schw_cart)
print("    ... Done.")

# --- 3. Generate GENERIC symbolic RHS expressions for MASSIVE geodesics ---
rhs_pos_massive = geodesic_pos_rhs_massive() 
rhs_vel_massive = geodesic_vel_rhs_massive()
all_rhs_expressions_massive = rhs_pos_massive + rhs_vel_massive

# --- 4. Generate symbolic recipes using the STABLE and original methods ---
print(" -> Generating symbolic recipes for initial conditions and conserved quantities...")
ut_expr_from_vel = ut_massive() # Keep original for the general C initializer
# *** THE FIX IS HERE: Call the new, numerically stable function ***
ut_expr, uphi_expr = symbolic_ut_uphi_from_r_stable()
print("    ... Initial condition recipes generated.")

# --- 5. Generate symbolic recipes for conserved quantities ---
E_expr = symbolic_energy()
Lx_expr, Ly_expr, Lz_expr = symbolic_L_components_cart()
Q_expr_kerr = symbolic_carter_constant_Q()
Q_expr_schw = Lx_expr**2 + Ly_expr**2 + Lz_expr**2
list_of_expressions_kerr = [E_expr, Lx_expr, Ly_expr, Lz_expr, Q_expr_kerr]
list_of_expressions_schw = [E_expr, Lx_expr, Ly_expr, Lz_expr, Q_expr_schw]
print("    ... Conservation recipes generated.")

<a id='generate_c_engines'></a>
# Step 6: C Code Generation - Physics "Engines" and "Workers"

This section marks our transition from pure symbolic mathematics to C code generation. The Python functions defined here are "meta-functions": their job is not to perform calculations themselves, but to **generate the C code** that will perform the calculations in the final compiled program.

We distinguish between several types of generated functions:
*   **Workers**: These are specialized functions that implement the physics for a *specific metric*. For example, `con_kerr_schild()` is a worker that only knows how to compute Christoffel symbols for the Kerr-Schild metric.
*   **Engines**: These are generic functions that implement physics equations valid for *any metric*. For example, `calculate_ode_rhs_massive()` is an engine that can compute the geodesic equations for any metric, as long as the Christoffel symbols are provided to it.
*   **Dispatchers**: These are simple functions that contain a `switch` statement to select and call the correct worker based on runtime parameters.
*   **Orchestrators**: These are high-level functions that manage the overall program flow, calling other engines, workers, and dispatchers to perform a complex task.

<a id='tier_4_workers'></a>
### 6.A.1: Metric and Connection "Workers"

The following functions generate the C **worker** functions. Each worker is a highly specialized piece of code that knows how to perform one specific calculation for one specific spacetime. They are generated directly from the complex symbolic expressions derived in Step 5.

This function, `g4DD_kerr_schild`, takes the symbolic expressions for the Kerr-Schild metric components (stored in the global variable `g4DD_kerr`) and converts them into a C function.

**How it Works:**
1.  It creates a Python list of the 10 unique symbolic metric components.
2.  It creates a corresponding list of C variable names where the results will be stored (e.g., `"metric->g00"`, `"metric->g01"`, etc.).
3.  It calls `nrpy.c_codegen.c_codegen()`, which takes the symbolic expressions and output variable names and returns a string of highly optimized C code. This C code automatically includes Common Subexpression Elimination (CSE) to avoid redundant calculations.
4.  Finally, it calls `nrpy.c_function.register_CFunction()` to store the generated C code string, along with metadata like the function name, parameters, and required header files, in a global dictionary. This dictionary will be used in the final step to write all the C files to disk.


In [ ]:
def g4DD_kerr_schild():
    """
    Generates and registers the C function to compute the Kerr-Schild
    metric components in Cartesian coordinates. This is the new unified worker.
    """
    print(" -> Generating C worker function: g4DD_kerr_schild()...")
    
    # We use the globally defined g4DD_kerr from the symbolic execution step
    list_of_g4DD_syms = []
    for i in range(4):
        for j in range(i, 4):
            list_of_g4DD_syms.append(g4DD_kerr[i][j])

    list_of_g4DD_C_vars = []
    for i in range(4):
        for j in range(i, 4):
            list_of_g4DD_C_vars.append(f"metric->g{i}{j}")

    includes = ["BHaH_defines.h"]
    desc = r"""@brief Computes the 10 unique components of the Kerr metric in Cartesian Kerr-Schild coords."""
    name = "g4DD_kerr_schild"
    # The state vector y now contains (t, x, y, z)
    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const double y[4], metric_struct *restrict metric"
   
    body = ccg.c_codegen(list_of_g4DD_syms, list_of_g4DD_C_vars, enable_cse=True)

    cfc.register_CFunction(
        includes=includes, desc=desc, name=name, params=params, body=body,
        include_CodeParameters_h=True
    )
    print("    ... g4DD_kerr_schild() registration complete.")


This function generates the C worker for the standard Schwarzschild metric in Cartesian coordinates. Its operation is identical to `g4DD_kerr_schild()`, but it uses the symbolic expressions from `g4DD_schw_cart` that were computed in Step 5. This provides an independent implementation that can be used for cross-validation against the Kerr-Schild code (when spin `a=0`).


In [ ]:
def g4DD_schwarzschild_cartesian():
    """
    Generates and registers the C function to compute the Schwarzschild
    metric components in standard Cartesian coordinates.
    """
    print(" -> Generating C worker function: g4DD_schwarzschild_cartesian()...")
    
    # Use the globally defined g4DD_schw_cart from the symbolic execution step
    list_of_g4DD_syms = []
    for i in range(4):
        for j in range(i, 4):
            list_of_g4DD_syms.append(g4DD_schw_cart[i][j])

    list_of_g4DD_C_vars = []
    for i in range(4):
        for j in range(i, 4):
            list_of_g4DD_C_vars.append(f"metric->g{i}{j}")

    includes = ["BHaH_defines.h"]
    desc = r"""@brief Computes the 10 unique components of the Schwarzschild metric in Cartesian coords."""
    name = "g4DD_schwarzschild_cartesian"
    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const double y[4], metric_struct *restrict metric"
   
    body = ccg.c_codegen(list_of_g4DD_syms, list_of_g4DD_C_vars, enable_cse=True)

    cfc.register_CFunction(
        includes=includes, desc=desc, name=name, params=params, body=body,
        include_CodeParameters_h=True
    )
    print("    ... g4DD_schwarzschild_cartesian() registration complete.")


This function generates the C worker for the Kerr-Schild Christoffel symbols. It follows the same pattern as the metric workers:
1.  It takes the 40 unique symbolic Christoffel symbol components from the global variable `Gamma4UDD_kerr`.
2.  It generates the corresponding C variable names (e.g., `"conn->Gamma4UDD012"`).
3.  It uses `nrpy.c_codegen.c_codegen()` to translate the complex symbolic formulas into optimized C code.
4.  It registers the resulting C function, `con_kerr_schild()`, which will be responsible for populating the `connection_struct` at runtime.

In [ ]:
def con_kerr_schild():
    """
    Generates and registers the C function to compute the Kerr-Schild Christoffel symbols.
    This is the new unified worker.
    """
    print(" -> Generating C worker function: con_kerr_schild()...")
    
    # We use the globally defined Gamma4UDD_kerr from the symbolic execution step
    list_of_Gamma_syms = []
    for i in range(4):
        for j in range(4):
            for k in range(j, 4):
                list_of_Gamma_syms.append(Gamma4UDD_kerr[i][j][k])

    conn_Gamma4UDD = ixp.declarerank3("conn->Gamma4UDD", dimension=4)
    list_of_Gamma_C_vars = []
    for i in range(4):
        for j in range(4):
            for k in range(j, 4):
                list_of_Gamma_C_vars.append(str(conn_Gamma4UDD[i][j][k]))

    includes = ["BHaH_defines.h"]
    desc = r"""@brief Computes the 40 unique Christoffel symbols for the Kerr metric in Kerr-Schild coords."""
    name = "con_kerr_schild"
    # The state vector y now contains (t, x, y, z)
    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const double y[4], connection_struct *restrict conn"

    body = ccg.c_codegen(list_of_Gamma_syms, list_of_Gamma_C_vars, enable_cse=True)
    
    cfc.register_CFunction(
        includes=includes, desc=desc, name=name, params=params, body=body,
        include_CodeParameters_h=True
    )
    print("    ... con_kerr_schild() registration complete.")

This function generates the C worker for the Christoffel symbols of the standard Schwarzschild metric in Cartesian coordinates. It uses the symbolic expressions from `Gamma4UDD_schw_cart` and follows the same code generation and registration process as `con_kerr_schild()`.


In [ ]:
def con_schwarzschild_cartesian():
    """
    Generates and registers the C function to compute the Schwarzschild Christoffel symbols
    in standard Cartesian coordinates.
    """
    print(" -> Generating C worker function: con_schwarzschild_cartesian()...")
    
    # Use the globally defined Gamma4UDD_schw_cart
    list_of_Gamma_syms = []
    for i in range(4):
        for j in range(4):
            for k in range(j, 4):
                list_of_Gamma_syms.append(Gamma4UDD_schw_cart[i][j][k])

    conn_Gamma4UDD = ixp.declarerank3("conn->Gamma4UDD", dimension=4)
    list_of_Gamma_C_vars = []
    for i in range(4):
        for j in range(4):
            for k in range(j, 4):
                list_of_Gamma_C_vars.append(str(conn_Gamma4UDD[i][j][k]))

    includes = ["BHaH_defines.h"]
    desc = r"""@brief Computes the unique Christoffel symbols for the Schwarzschild metric in Cartesian coords."""
    name = "con_schwarzschild_cartesian"
    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const double y[4], connection_struct *restrict conn"

    body = ccg.c_codegen(list_of_Gamma_syms, list_of_Gamma_C_vars, enable_cse=True)

    cfc.register_CFunction(
        includes=includes, desc=desc, name=name, params=params, body=body,
        include_CodeParameters_h=True
    )
    print("    ... con_schwarzschild_cartesian() registration complete.")


### 6.A.2: Metric and Connection "Dispatchers"

Dispatchers are simple C functions that act as switchboards. Based on a runtime parameter (like `metric_choice`), they call the appropriate specialized worker function. This separates the control flow from the physics implementation, making the code cleaner and easier to extend with new metrics in the future.

The `g4DD_metric()` function generates a C dispatcher that computes the metric components.

**How the C Function Works:**
The generated C function `g4DD_metric()` contains a `switch` statement that checks the `metric->type` enum.
*   If the type is `Kerr` or `Schwarzschild` (which uses the Kerr-Schild implementation with `a=0`), it calls the `g4DD_kerr_schild()` worker.
*   If the type is `Schwarzschild_Standard`, it calls the `g4DD_schwarzschild_cartesian()` worker.
This allows the rest of the C code to remain agnostic to the specific metric being used; it can simply call `g4DD_metric()` and be assured of getting the correct result.


In [ ]:
def g4DD_metric():
    """
    Generates and registers the C function g4DD_metric(), which serves as a
    dispatcher to call the appropriate metric-specific worker function.
    """
    print(" -> Generating C dispatcher function: g4DD_metric()...")
    
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h"]
    desc = r"""@brief Dispatcher to compute the 4-metric g_munu for the chosen metric."""
    name = "g4DD_metric"
    # The signature is now coordinate-aware, but the y vector is always Cartesian here.
    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const metric_params *restrict metric, const double y[8], metric_struct *restrict metric_out"
    
    body = r"""
    // The state vector y_pos contains only the position coordinates.
    const double y_pos[4] = {y[0], y[1], y[2], y[3]};

    // This switch statement chooses which "worker" function to call
    // based on the metric type provided.
    switch(metric->type) {
        case Schwarzschild:
        case Kerr:
            // For Kerr or Schwarzschild in KS coords, call the unified Kerr-Schild C function.
            g4DD_kerr_schild(commondata, params, y_pos, metric_out);
            break;
        // <-- MODIFIED: Call the new Cartesian worker
        case Schwarzschild_Standard:
            g4DD_schwarzschild_cartesian(commondata, params, y_pos, metric_out);
            break;
        case Numerical:
            printf("Error: Numerical metric not supported yet.\n");
            exit(1);
            break;
        default:
            printf("Error: MetricType %d not supported in g4DD_metric() yet.\n", metric->type);
            exit(1);
            break;
    }
"""
    
    cfc.register_CFunction(includes=includes, desc=desc, name=name, params=params, body=body)
    print("    ... g4DD_metric() registration complete.")

The `connections()` function generates the C dispatcher for the Christoffel symbols. Its structure and purpose are identical to `g4DD_metric()`. The generated C function `connections()` contains a `switch` statement on `metric->type` and calls the appropriate worker function (`con_kerr_schild()` or `con_schwarzschild_cartesian()`) to compute and store the Christoffel symbols in the `connection_struct`.

In [ ]:
def connections():
    """
    Generates and registers the C dispatcher for Christoffel symbols.
    """
    print(" -> Generating C dispatcher: connections()...")

    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "stdio.h", "stdlib.h"]
    desc = r"""@brief Dispatcher to compute Christoffel symbols for the chosen metric."""
    
    name = "connections"
    cfunc_type = "void" 
    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const metric_params *restrict metric, const double y[8], connection_struct *restrict conn"

    body = r"""
    // The state vector y_pos contains only the position coordinates.
    const double y_pos[4] = {y[0], y[1], y[2], y[3]};

    // This switch statement chooses which "worker" function to call
    // based on the metric type provided.
    switch(metric->type) {
        case Schwarzschild:
        case Kerr:
            con_kerr_schild(commondata, params, y_pos, conn);
            break;
        // <-- MODIFIED: Call the new Cartesian worker
        case Schwarzschild_Standard:
            con_schwarzschild_cartesian(commondata, params, y_pos, conn);
            break;
        case Numerical:
            printf("Error: Numerical metric not supported yet.\n");
            exit(1);
            break;
        default:
            printf("Error: MetricType %d not supported yet.\n", metric->type);
            exit(1);
            break;
    }
"""

    cfc.register_CFunction(
        includes=includes, desc=desc, cfunc_type=cfunc_type,
        name=name, params=params, body=body
    )
    print("    ... connections() registration complete.")

### 6.A.3: Specialized Physics Helpers

This section contains generators for specialized "helper" functions that perform a single, crucial physics calculation.

The `calculate_ut_uphi_from_r()` function generates the C implementation of the numerically stable recipe for finding the initial 4-velocity components $u^t$ and $u^\phi$ for a circular orbit. It takes the symbolic expressions `ut_expr` and `uphi_expr` (which were derived using the stable method in Step 3.d) and uses `nrpy.c_codegen.c_codegen()` to convert them into an efficient C function. This C function will be called by the initial condition orchestrators to set up the particle disk.


In [ ]:
def calculate_ut_uphi_from_r():
    """
    Generates a C helper function to compute u^t and u^phi from a radius
    using a numerically stable recipe.
    """
    print(" -> Generating C engine: calculate_ut_uphi_from_r() [STABLE VERSION]...")
    
    includes = ["BHaH_defines.h"]
    desc = r"""@brief Computes u^t and u^phi for a circular orbit at a given radius using a numerically stable method."""
    name = "calculate_ut_uphi_from_r"
    params = "const double r_initial, const commondata_struct *restrict commondata, const params_struct *restrict params, double *ut, double *uphi"
    
    # The global ut_expr and uphi_expr are now the stable versions
    body = ccg.c_codegen(
        [ut_expr, uphi_expr],
        ["*ut", "*uphi"],
        enable_cse=True
    )
    
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body=body,
        include_CodeParameters_h=True
    )
    print("    ... calculate_ut_uphi_from_r() registration complete.")

<a id='tier_3_engines'></a>
### 6.B.1: The Geodesic Equation Engine

This function generates the core physics engine of the integrator. It is completely generic and has no knowledge of any specific spacetime.

The `calculate_ode_rhs_massive()` function takes the generic symbolic expressions for the geodesic equations of motion (stored in `all_rhs_expressions_massive`) and generates the C engine that computes the right-hand side of the ODE system, $d\vec{y}/d\tau$.

**How the C Function Works:**
The generated C function `calculate_ode_rhs_massive()` takes the current state vector `y[8]` and a pointer to a `connection_struct` (which has already been filled by the `connections()` dispatcher) as input. It then executes the C code corresponding to the generic geodesic equations:
$$ \frac{dx^{\alpha}}{d\tau} = u^{\alpha} $$
$$ \frac{du^{\alpha}}{d\tau} = -\Gamma^{\alpha}_{\mu\nu} u^{\mu} u^{\nu} $$
The values for $\Gamma^{\alpha}_{\mu\nu}$ are read directly from the input `connection_struct`. The function calculates the 8 derivatives and writes them to the output array `rhs_out[8]`. This engine is the heart of the physics simulation, called at every step of the ODE integration.

In [ ]:
def calculate_ode_rhs_massive():
    """
    Generates the C engine to calculate the RHS of the 8 massive particle ODEs.
    """
    includes = ["BHaH_defines.h"]
    desc = r"""@brief Calculates the right-hand sides (RHS) of the 8 massive particle geodesic ODEs.
    
    This is a generic engine that implements the geodesic equation using pre-computed
    Christoffel symbols from the connection_struct.
    
    @param[in]  y         The 8-component state vector [t, x, y, z, u^t, u^x, u^y, u^z].
    @param[in]  conn      A pointer to the connection_struct holding the Christoffel symbols.
    @param[out] rhs_out   A pointer to the 8-component output array for the RHS results."""
    name = "calculate_ode_rhs_massive"
    params = "const double y[8], const connection_struct *restrict conn, double rhs_out[8]"
    
    rhs_output_vars = [f"rhs_out[{i}]" for i in range(8)]
    body = ccg.c_codegen(all_rhs_expressions_massive, rhs_output_vars)

    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body=body
    )

### 6.B.2: The Conservation Check Engine

This function generates the C engine used for validation. It computes the conserved quantities (Energy, Angular Momentum, and Carter Constant) from a particle's state vector. By comparing these values at the start and end of an integration, we can measure the numerical accuracy of the solver.

**How it Works:**
This function generates a C function `check_conservation_massive()` that contains two distinct blocks of code, one for the Kerr metric and one for the Schwarzschild metric.
1.  It uses `nrpy.c_codegen.c_codegen()` on the symbolic expressions `list_of_expressions_kerr` (which includes the Carter Constant `Q_expr_kerr`) to generate the C code for the Kerr case.
2.  It does the same for `list_of_expressions_schw` (which uses $L^2$ instead of Q) to generate the code for the Schwarzschild case.
3.  It embeds these two C code blocks inside an `if/else` statement in the final C function body. At runtime, the C function will check the `metric_params_in->type` and execute the appropriate block of code to calculate the conserved quantities.


In [ ]:
def check_conservation_massive():
    """
    Generates the C function `check_conservation_massive`.
    """
    print(" -> Generating C engine: check_conservation_massive()...")

    output_vars_kerr = ["*E", "*Lx", "*Ly", "*Lz", "*Q"]
    output_vars_schw = ["*E", "*Lx", "*Ly", "*Lz", "*Q"]

    body_C_code_kerr = ccg.c_codegen(list_of_expressions_kerr, output_vars_kerr, enable_cse=True, include_braces=False)
    body_C_code_schw = ccg.c_codegen(list_of_expressions_schw, output_vars_schw, enable_cse=True, include_braces=False)

    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h"]
    desc = r"""@brief Computes conserved quantities (E, L_i, Q/L^2) for a given massive particle state vector."""
    name = "check_conservation_massive"
    params = """const commondata_struct *restrict commondata,
        const params_struct *restrict params,
        const metric_params *restrict metric_params_in,
        const double y[8], 
        double *E, double *Lx, double *Ly, double *Lz, double *Q"""
        
    body = r"""
    // Unpack parameters from commondata struct that are needed symbolically
    const REAL a_spin = commondata->a_spin;

    metric_struct* metric = (metric_struct*)malloc(sizeof(metric_struct));
    g4DD_metric(commondata, params, metric_params_in, y, metric);

    if (metric_params_in->type == Kerr) {
        """ + body_C_code_kerr + r"""
    } else { // Both Schwarzschild types are now Cartesian
        """ + body_C_code_schw + r"""
    }
    
    free(metric);
    """

    cfc.register_CFunction(
        includes=includes, desc=desc, cfunc_type="void",
        name=name, params=params, body=body
    )
    print(f"    ... {name}() registration complete.")

### 6.B.3: The GSL Wrapper Engine

This function generates the crucial "bridge" or "wrapper" function that allows our custom C code to communicate with the external GNU Scientific Library (GSL) ODE solver.

**Why it's Needed:**
The GSL library provides powerful, general-purpose ODE solvers. To use them, you must provide a C function that calculates the right-hand side of your ODE system and matches a specific function signature required by GSL:
`int function_name(double t, const double y[], double f[], void *params)`
*   `t`: The independent variable (in our case, proper time `τ`).
*   `y[]`: The current state vector.
*   `f[]`: The output array where the derivatives (the RHS) must be written.
*   `params`: A generic `void` pointer used to pass any extra information your function needs.

**How it Works:**
The `ode_gsl_wrapper_massive()` function generates a C function with exactly this signature. Inside the C function, it:
1.  Unpacks the `void *params` pointer into our custom `gsl_params` struct, which gives it access to the black hole parameters and metric choice.
2.  Calls the `g4DD_metric()` and `connections()` dispatchers to compute the metric and Christoffel symbols at the particle's current position.
3.  Calls the `calculate_ode_rhs_massive()` engine to compute the physical derivatives $d\vec{y}/d\tau$.
4.  Writes the results into the output array `f[]` that GSL provided.
5.  Returns `GSL_SUCCESS` to signal that the step was completed correctly.

This wrapper function is the glue that connects the generic GSL solver to our specific physics implementation.


In [ ]:
def ode_gsl_wrapper_massive():
    """
    Generates the C function that acts as a bridge between the GSL ODE
    solver and our project-specific physics functions.
    """
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "gsl/gsl_errno.h"]
    desc = r"""@brief GSL wrapper for the massive particle geodesic ODEs.
    
    This function matches the signature required by the GSL ODE solver. It unpacks
    the gsl_params carrier struct and calls the dispatchers for the metric and
    Christoffel symbols, before finally calling the RHS engine.
    
    @param[in]  t      The current value of the independent variable (proper time τ). Unused.
    @param[in]  y      The current 8-component state vector.
    @param[in]  params A generic void pointer to our gsl_params carrier struct.
    @param[out] f      A pointer to the 8-component output array where GSL expects the RHS results."""
    cfunc_type = "int"
    name = "ode_gsl_wrapper_massive"
    params = "double t, const double y[8], double f[8], void *params"
    
    body = r"""
        (void)t; // Proper time 't' is not explicitly used in the RHS expressions.
        
        // Unpack the carrier struct to access simulation parameters and metric choice.
        gsl_params *gsl_parameters = (gsl_params *)params;
        
        // Declare structs to hold metric and connection values.
        metric_struct g4DD;
        connection_struct conn;
        
        // Call dispatchers to compute the metric and Christoffel symbols at the current position y.
        // Note: The y array is 8D, but these functions only need the first 4 position components.
        g4DD_metric(gsl_parameters->commondata, gsl_parameters->params, gsl_parameters->metric, y, &g4DD);
        connections(gsl_parameters->commondata, gsl_parameters->params, gsl_parameters->metric, y, &conn);
        
        // Call the engine to compute the RHS of the ODEs.
        calculate_ode_rhs_massive(y, &conn, f);
        
        return GSL_SUCCESS;
    """
    
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        cfunc_type=cfunc_type,
        name=name, 
        params=params,
        body=body
    )

# <!-- MARKDOWN FOR: set_initial_conditions_massive -->
"""
<a id='tiers_1_2_orchestrators'></a>
### 6.C.1: Initial Condition Orchestrators

The following functions generate high-level C orchestrators responsible for creating the initial distribution of particles for a production run. They are called by the main simulation loop based on the `initial_conditions_type` parameter.

The `set_initial_conditions_massive()` function generates a C helper that takes a particle's initial position and computes the full 8-component state vector for a stable, circular, equatorial orbit.

**How the C Function Works:**
The generated C function is a direct C implementation of the numerically stable three-step method described in Step 3.d.
1.  It calculates the coordinate angular velocity $\Omega = d\phi/dt$.
2.  It calculates the necessary Boyer-Lindquist metric components ($g_{tt}, g_{t\phi}, g_{\phi\phi}$) at the given radius.
3.  It solves for $u^t = dt/d\tau$ using the stable formula and then finds $u^\phi = d\phi/d\tau = \Omega u^t$.
4.  Finally, it assembles the full 8-component state vector in Cartesian coordinates. This involves transforming the 4-velocity component $u^\phi$ into Cartesian 4-velocity components:
    $$ u^x = \frac{dx}{d\tau} = \frac{d(r\cos\phi)}{d\tau} = -r\sin\phi \frac{d\phi}{d\tau} = -y u^\phi $$
    $$ u^y = \frac{dy}{d\tau} = \frac{d(r\sin\phi)}{d\tau} = r\cos\phi \frac{d\phi}{d\tau} = x u^\phi $$
The final state vector `y_out[8]` is then ready to be passed to the integrator.
"""

In [ ]:
def set_initial_conditions_massive():
    """
    Generates the C engine to set the full initial 8-component state vector.
    
    VERSION 2: This version implements the NUMERICALLY STABLE method for
    calculating u^t, as detailed in the Gemini report. It computes the
    Boyer-Lindquist metric components and Omega internally to avoid
    catastrophic cancellation.
    """
    print(" -> Generating C engine: set_initial_conditions_massive() [STABLE VERSION]...")
    
    includes = ["BHaH_defines.h", "<math.h>"]
    desc = r"""@brief Sets the initial 8-component state vector for a massive particle using a numerically stable method."""
    name = "set_initial_conditions_massive"
    params = "const particle_initial_state_t *restrict initial_state, const commondata_struct *restrict commondata, double y_out[8]"

    body = r"""
    // Unpack parameters for clarity
    const double M = commondata->M_scale;
    const double a = commondata->a_spin;
    
    // Unpack initial position
    const double x = initial_state->pos[1];
    const double y = initial_state->pos[2];
    const double z = initial_state->pos[3];
    const double r = sqrt(x*x + y*y + z*z);

    // --- Numerically Stable Method to find u^t and u^phi ---
    
    // Step 1: Calculate Omega = d(phi)/dt
    const double Omega = sqrt(M) / (pow(r, 1.5) + a * sqrt(M));

    // Step 2: Calculate Boyer-Lindquist metric components in the equatorial plane
    const double g_tt = -(1.0 - 2.0*M/r);
    const double g_tphi = -2.0*a*M/r;
    const double g_phiphi = r*r + a*a + (2.0*M*a*a)/r;

    // Step 3: Solve for u^t using the normalization condition
    const double ut_inv_denom = g_tt + 2.0*g_tphi*Omega + g_phiphi*Omega*Omega;
    if (ut_inv_denom >= 0) {
        // This indicates an unstable or invalid orbit. Set state to NaN.
        for(int i=0; i<8; i++) y_out[i] = NAN;
        return;
    }
    const double ut = sqrt(-1.0 / ut_inv_denom);
    const double uphi = Omega * ut;

    // --- Assemble the final 8-component state vector ---
    y_out[0] = initial_state->pos[0]; // t
    y_out[1] = x;
    y_out[2] = y;
    y_out[3] = z;
    
    y_out[4] = ut;
    // Transform u^phi to Cartesian u^x, u^y
    y_out[5] = -y * uphi;
    y_out[6] =  x * uphi;
    y_out[7] = 0.0; // u^z is zero for equatorial orbits
    """
    
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body=body
    )


The `generate_disk_initial_conditions()` function generates a C orchestrator that creates a simple, axisymmetric Keplerian disk of particles.

**How the C Function Works:**
The generated C function `generate_disk_initial_conditions()` populates a large array of initial state vectors. It does this by looping through a grid of radii `r` and azimuthal angles `phi`. At each `(r, phi)` point, it:
1.  Calls the `calculate_ut_uphi_from_r()` helper function to get the correct 4-velocity components for a stable circular orbit at that radius.
2.  Converts the polar position and velocity into Cartesian components.
3.  Stores the resulting 8-component state vector in the main array.
The function returns the total number of particles created.


In [ ]:
def generate_disk_initial_conditions():
    """
    Generates a C function that programmatically creates the initial conditions
    for a Keplerian disk.
    """
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h"]
    desc = r"""@brief Generates the complete y[8] initial state for all particles in a Keplerian disk."""
    cfunc_type = "int"
    name = "generate_disk_initial_conditions"

    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, double *restrict y_initial_states"
    

    body = r"""
    int particle_count = 0;
    const double dr = (commondata->disk_num_r > 1) ? (commondata->disk_r_max - commondata->disk_r_min) / (commondata->disk_num_r - 1) : 0;

    for (int i = 0; i < commondata->disk_num_r; i++) {
        const double r = commondata->disk_r_min + i * dr;
        
        const int num_phi_at_r = (commondata->disk_num_phi > 1) ? (int)(commondata->disk_num_phi * (r / commondata->disk_r_max)) : 1;
        if (num_phi_at_r == 0) continue;
        const double dphi = 2.0 * M_PI / num_phi_at_r;
        
        double ut_at_r, uphi_at_r;
        // The call is now valid because 'params' is available in this function's scope.
        calculate_ut_uphi_from_r(r, commondata, params, &ut_at_r, &uphi_at_r);

        for (int j = 0; j < num_phi_at_r; j++) {
            const double phi = j * dphi;
            const double cos_phi = cos(phi);
            const double sin_phi = sin(phi);
            
            double *y = &y_initial_states[particle_count * 8];
            
            y[0] = 0.0;
            y[1] = r * cos_phi;
            y[2] = r * sin_phi;
            y[3] = 0.0;
            
            y[4] = ut_at_r;
            y[5] = -(r * sin_phi) * uphi_at_r;
            y[6] =  (r * cos_phi) * uphi_at_r;
            y[7] = 0.0;
            
            particle_count++;
        }
    }
    return particle_count;
    """
    
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        cfunc_type=cfunc_type,
        name=name,
        params=params,
        body=body
    )

The `generate_spiral_galaxy_initial_conditions()` function generates a C orchestrator that places particles along logarithmic spiral arms to create a "grand design" spiral galaxy pattern.

**How the C Function Works:**
Instead of a uniform grid, this function uses a random placement strategy. For each particle, it:
1.  Randomly chooses a radius `r` within the disk's extent.
2.  Calculates a base angle $\theta_{\text{base}}$ using the logarithmic spiral formula:
    $$ \theta_{\text{base}} = \frac{1}{b} \ln\left(\frac{r}{r_{\min}}\right) $$
    where `b` is the `arm_tightness` parameter.
3.  Randomly assigns the particle to one of the spiral arms by adding an angular offset.
4.  Adds a small amount of random scatter to the final angle `phi` to give the arms some thickness.
5.  Once the final `(r, phi)` position is determined, it calls `calculate_ut_uphi_from_r()` to set the particle on a stable circular orbit, just like the other initial condition generators.

In [ ]:
def generate_spiral_galaxy_initial_conditions():
    """
    Generates a C function that programmatically creates the initial conditions
    for a spiral galaxy disk.
    
    UPDATED to use runtime parameters from the commondata struct to control
    the number and tightness of the spiral arms.
    """
    print(" -> Generating C function: generate_spiral_galaxy_initial_conditions()...")
    
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "<math.h>", "<stdlib.h>"] # Added stdlib.h for rand()
    desc = r"""@brief Generates the complete y[8] initial state for all particles in a spiral galaxy disk."""
    cfunc_type = "int"
    name = "generate_spiral_galaxy_initial_conditions"

    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, double *restrict y_initial_states"
    
    body = r"""
    int particle_count = 0;
    const int num_particles_total = commondata->disk_num_r * commondata->disk_num_phi;
    
    // --- Spiral galaxy parameters are now read from the commondata struct ---
    const int num_arms = commondata->spiral_galaxy_num_arms;
    const double arm_tightness = commondata->spiral_galaxy_arm_tightness;

    // Seed the random number generator for reproducibility if needed.
    // For true randomness on each run, you could seed with time(NULL).
    srand(42); 

    for (int i = 0; i < num_particles_total; i++) {
        // --- Particle Placement Logic (Unchanged, but now uses parameters) ---
        
        const double r = commondata->disk_r_min + (commondata->disk_r_max - commondata->disk_r_min) * sqrt((double)rand() / RAND_MAX);

        // The base angle for this radius from the logarithmic spiral formula.
        const double theta_base = (1.0 / arm_tightness) * log(r / commondata->disk_r_min);

        const int arm_index = rand() % num_arms;
        const double arm_offset = (2.0 * M_PI / num_arms) * arm_index;

        const double phi_spread = (M_PI / num_arms) * 0.2 * ((double)rand() / RAND_MAX - 0.5);
        
        const double phi = theta_base + arm_offset + phi_spread;
        
        // --- The rest of this logic is IDENTICAL to the original ---
        
        const double cos_phi = cos(phi);
        const double sin_phi = sin(phi);
        
        double ut_at_r, uphi_at_r;
        calculate_ut_uphi_from_r(r, commondata, params, &ut_at_r, &uphi_at_r);

        double *y = &y_initial_states[particle_count * 8];
        
        y[0] = 0.0;
        y[1] = r * cos_phi;
        y[2] = r * sin_phi;
        y[3] = 0.0;
        y[4] = ut_at_r;
        y[5] = -(r * sin_phi) * uphi_at_r;
        y[6] =  (r * cos_phi) * uphi_at_r;
        y[7] = 0.0;
        
        particle_count++;
    }
    return particle_count;
    """
    
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        cfunc_type=cfunc_type,
        name=name,
        params=params,
        body=body
    )
    print("    ... generate_spiral_galaxy_initial_conditions() registration complete.")

The `generate_barred_flocculent_spiral_ic()` function generates the most complex initial condition orchestrator, creating a realistic barred spiral galaxy with clumpy, "flocculent" arms.

**How the C Function Works:**
This function uses a technique called **rejection sampling** to create the complex geometry. The process is as follows:
1.  A trial particle position `(x, y)` is generated randomly and uniformly within the disk annulus.
2.  The code checks which region the particle falls into: the central bulge (a circle), the bar (a rectangle), or the outer disk.
3.  Based on the region, an "acceptance probability" is calculated. This probability is highest in the bulge and bar, and in the outer disk, it's modulated by a function that describes the spiral arms and adds clumpy noise.
4.  A random number is generated. If it is less than the acceptance probability, the particle's position is kept. If not, the position is rejected, and the process repeats from step 1 until an accepted position is found.

This method allows for the creation of arbitrary, non-uniform density distributions. Despite the complex placement, every accepted particle is still placed on a stable, circular Keplerian orbit by calling `calculate_ut_uphi_from_r()`.

In [ ]:
def generate_barred_flocculent_spiral_ic():
    """
    Generates a C function that programmatically creates the initial conditions
    for a realistic barred spiral galaxy with clumpy, flocculent arms.
    
    This function uses rejection sampling to place particles in one of three regions:
    a central bulge, a rectangular bar, or flocculent spiral arms.
    
    Despite the complex geometry, all particles are placed on stable, circular
    Keplerian orbits, satisfying the project constraints.
    """
    print(" -> Generating C function: generate_barred_flocculent_spiral_ic()...")
    
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "<math.h>", "<stdlib.h>", "<time.h>"]
    desc = r"""@brief Generates the initial state for all particles in a barred flocculent spiral galaxy."""
    cfunc_type = "int"
    name = "generate_barred_flocculent_spiral_ic"

    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, double *restrict y_initial_states"
    


    body = r"""
        // Seed the random number generator. A unique seed is needed for each thread in parallel.
        // We will use the particle index + a base seed.
        unsigned int seed = 42;

        int particle_count = 0;
        const int num_particles_total = commondata->disk_num_r * commondata->disk_num_phi;

        // Unpack geometry parameters from commondata for clarity
        const double r_min = commondata->disk_r_min;
        const double r_max = commondata->disk_r_max;
        const double bar_len = commondata->bar_length;
        const double bar_width = commondata->bar_length * commondata->bar_aspect_ratio;
        const double bulge_rad = commondata->bulge_radius;
        const int num_arms = commondata->spiral_galaxy_num_arms;
        const double arm_tightness = commondata->spiral_galaxy_arm_tightness;

        for (int i = 0; i < num_particles_total; i++) {
            double r, phi, x, y;
            
            // Use the thread-safe random number generator
            seed += i;

            // --- REJECTION SAMPLING LOOP ---
            while (1) {
                // Generate a trial particle position uniformly in the disk annulus
                double random_val = (double)rand_r(&seed) / RAND_MAX;
                r = sqrt(random_val * (r_max*r_max - r_min*r_min) + r_min*r_min);
                phi = 2.0 * M_PI * ((double)rand_r(&seed) / RAND_MAX);
                x = r * cos(phi);
                y = r * sin(phi);

                double acceptance_prob = 0.0;

                // Region 1: Central Bulge
                if (r < bulge_rad) {
                    acceptance_prob = commondata->bulge_density_factor * commondata->arm_particle_density;
                }
                // Region 2: Central Bar
                else if (fabs(x) < bar_len / 2.0 && fabs(y) < bar_width / 2.0) {
                    acceptance_prob = commondata->bar_density_factor * commondata->arm_particle_density;
                }
                // Region 3: Flocculent Spiral Arms
                else {
                    // Optimization: Do a cheap check first. If the base probability fails,
                    // no need to do expensive log/exp/cos calls.
                    if (((double)rand_r(&seed) / RAND_MAX) < commondata->arm_particle_density) {
                        // --- CORRECTED ARM PROFILE LOGIC ---
                        double theta_base = (1.0 / arm_tightness) * log(r / r_min);
                        double delta_phi_raw = phi - theta_base;
                        double angle_between_arms = 2.0 * M_PI / num_arms;
                        double delta_phi_folded = fmod(delta_phi_raw, angle_between_arms);

                        if (delta_phi_folded > 0.5 * angle_between_arms) {
                            delta_phi_folded -= angle_between_arms;
                        } else if (delta_phi_folded < -0.5 * angle_between_arms) {
                            delta_phi_folded += angle_between_arms;
                        }
                        
                        double arm_profile = exp(-0.5 * SQR(delta_phi_folded / (arm_tightness * 0.5)));
                        
                        // Flocculent/clumpy modulation
                        double clump_modulation = cos(commondata->arm_clumpiness_factor * (theta_base - phi));
                        clump_modulation *= cos(num_arms * phi * commondata->arm_clump_size);
                        
                        acceptance_prob = arm_profile + 0.5 * clump_modulation;
                    }
                }

                if (((double)rand_r(&seed) / RAND_MAX) < acceptance_prob) {
                    break; // Accept this particle's position
                }
            } // End of rejection sampling while loop

            // --- Velocity Assignment ---
            double ut_at_r, uphi_at_r;
            calculate_ut_uphi_from_r(r, commondata, params, &ut_at_r, &uphi_at_r);

            // --- Store the Full 8-Component State Vector ---
            double *y_state = &y_initial_states[particle_count * 8];
            
            y_state[0] = 0.0;
            y_state[1] = x;
            y_state[2] = y;
            y_state[3] = 0.0;
            y_state[4] = ut_at_r;
            y_state[5] = -y * uphi_at_r;
            y_state[6] =  x * uphi_at_r;
            y_state[7] = 0.0;
            
            particle_count++;
        }
        
        return particle_count;
    """
    
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        cfunc_type=cfunc_type,
        name=name,
        params=params,
        body=body
    )

### 6.C.2: The Integration Orchestrators

These functions generate the C code that manages the actual time-evolution of particles. They are the highest-level "engine room" of the simulation.

The `integrate_single_particle()` function generates the C orchestrator for integrating a single particle in a production run.

**Why it's Needed:**
This function is the workhorse of the main simulation loop. Its crucial task is to evolve a particle from the current coordinate time `t_start` to the exact time of the next snapshot, `t_end`.

**How the C Function Works:**
1.  **GSL Driver Setup**: It allocates a GSL "driver" (`gsl_odeiv2_driver_alloc_y_new`). This is a high-level object that manages an adaptive step-size integrator (in this case, RKF45).
2.  **Set up ODE System**: It points the GSL system to our `ode_gsl_wrapper_massive` function.
3.  **Call `gsl_odeiv2_driver_apply`**: This is the key step. It tells the GSL driver: "Start with the state `y_in_out` at time `t_start` and integrate forward until the independent variable reaches exactly `t_end`." The driver automatically handles all the internal time steps and interpolation needed to hit the target time precisely. This is how all particles in the simulation are synchronized to the same snapshot time.
4.  **Termination Checks**: After the integration, it checks if the particle has fallen into the black hole, escaped the simulation domain, or developed a runaway (unphysically large) velocity. If any of these occur, it returns a failure code.

In [ ]:
def integrate_single_particle():
    """
    Generates the main C integration loop for a single massive particle.
    This high-performance "production" version uses the GSL driver to ensure
    the state is returned at the exact requested snapshot times.
    """
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "gsl/gsl_errno.h", "gsl/gsl_odeiv2.h", "<math.h>"]
    desc = r"""@brief Integrates a single massive particle path between two times.
    
    This function uses the GSL driver, which internally uses an adaptive
    step-size algorithm (RKF45) to evolve the particle's state vector y_in_out
    from t_start to t_end, returning the state at the precise t_end.
    
    @param[in]      commondata  Pointer to commondata struct.
    @param[in]      params      Pointer to params_struct.
    @param[in]      metric      Pointer to metric_params struct.
    @param[in]      t_start     The starting proper time (τ) for the integration.
    @param[in]      t_end       The ending proper time (τ) for the integration.
    @param[in,out]  y_in_out    The 8-component state vector. Input is the state at t_start, output is the state at t_end.
    
    @return 0 on success, 1 on GSL failure.
    """
    cfunc_type = "int"
    name = "integrate_single_particle"
    params = """const commondata_struct *restrict commondata,
    const params_struct *restrict params,
    const metric_params *restrict metric,
    const double t_start, const double t_end,
    double y_in_out[8]"""

    body = r"""
    // Define the GSL ODE system
    gsl_params gsl_parameters = {commondata, params, metric};
    gsl_odeiv2_system sys = {ode_gsl_wrapper_massive, NULL, 8, &gsl_parameters};
    
    // Set up the GSL driver
    gsl_odeiv2_driver *d = gsl_odeiv2_driver_alloc_y_new(
        &sys, gsl_odeiv2_step_rkf45, 1e-6, 1e-11, 1e-11);
    
    double t = t_start;
    
    // The driver will take internal steps to reach t_end precisely.
    int status = gsl_odeiv2_driver_apply(d, &t, t_end, y_in_out);

    if (status != GSL_SUCCESS) {
        // Don't print an error here, as the orchestrator will check the status.
        // Just free memory and return the failure code.
        gsl_odeiv2_driver_free(d);
        return 1; // Return failure code
    }

    // Robustness check after the step
    const double r_sq = y_in_out[1]*y_in_out[1] + y_in_out[2]*y_in_out[2] + y_in_out[3]*y_in_out[3];
    const double r_horizon = commondata->M_scale * (1.0 + sqrt(1.0 - commondata->a_spin*commondata->a_spin));

    if (r_sq < r_horizon*r_horizon || r_sq > r_escape*r_escape || fabs(y_in_out[4]) > ut_max) {
        gsl_odeiv2_driver_free(d);
        return 1; // Return failure code
    }

    gsl_odeiv2_driver_free(d);
    return 0; // Return success code
    """
    
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        cfunc_type=cfunc_type,
        name=name,
        params=params,
        body=body, 
        include_CodeParameters_h=True
    )

The `integrate_single_particle_DEBUG()` function generates a C orchestrator specifically for debugging and validation runs.

**Why it's Needed:**
For validation, we need to see the full, detailed path of a single particle, not just its state at discrete snapshot times. This function provides that capability.

**How it Works:**
Instead of the high-level GSL driver, this function uses a lower-level GSL interface, `gsl_odeiv2_evolve_apply`.
1.  **File Setup**: It opens a text file, `massive_particle_path.txt`, and writes a header.
2.  **Integration Loop**: It enters a `for` loop that takes many small steps in proper time `τ`.
3.  **`gsl_odeiv2_evolve_apply`**: Inside the loop, it calls this function to take a single adaptive step forward in proper time.
4.  **Write to File**: After each successful step, it writes the full 8-component state vector to the text file.
5.  **Termination Checks**: It performs the same termination checks as the production integrator and breaks the loop if the particle's trajectory ends. These checks are:
    *   `r_sq < r_horizon*r_horizon`: The particle's squared radius `r_sq` is less than the squared event horizon radius `r_horizon`. The event horizon is the "point of no return" for a black hole, given by $r_H = M + \sqrt{M^2 - a^2}$. Any particle crossing this boundary is considered captured.
    *   `r_sq > r_escape*r_escape`: The particle has moved beyond a large, predefined `r_escape` radius and is considered to have escaped the system.
    *   `fabs(y_c[4]) > ut_max`: The time component of the 4-velocity, $u^t = y_c[4] = dt/d\tau$, has grown excessively large. This is related to the particle's energy and represents the rate at which coordinate time passes relative to the particle's own proper time. An extremely large value is unphysical for a stable orbit and usually indicates a numerical instability (a "runaway" error), so the integration is stopped.
    *   `fabs(y_c[0]) > t_max_integration`: The coordinate time $t = y_c[0]$ has exceeded the maximum allowed simulation time. This prevents runaway simulations.

This process generates a detailed log of the particle's path, which can then be analyzed by the companion visualization notebook to validate the integrator's accuracy.

In [ ]:
def integrate_single_particle_DEBUG():
    """
    Generates a DEBUG version of the integrator that writes the full trajectory
    of a single massive particle to a text file for validation.
    """
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "gsl/gsl_errno.h", "gsl/gsl_odeiv2.h"]
    desc = r"""@brief DEBUG integrator for a single massive particle.
    
    This function integrates the path of a single particle and writes the full
    8-component state vector at each step to 'massive_particle_path.txt'.
    It also prints progress to the console and checks for termination conditions.
    
    @param[in]  commondata  Pointer to the commondata_struct.
    @param[in]  params      Pointer to the params_struct.
    @param[in]  metric      Pointer to the metric_params struct.
    @param[in]  start_y     The 8-component initial state vector.
    @param[out] final_y_state The 8-component final state vector upon termination."""
    cfunc_type = "void"
    name = "integrate_single_particle_DEBUG"
    params = """const commondata_struct *restrict commondata,
    const params_struct *restrict params,
    const metric_params *restrict metric,
    const double start_y[8],
    double final_y_state[8]"""

    body = r"""
    // GSL Setup
    const gsl_odeiv2_step_type * T = gsl_odeiv2_step_rkf45;
    gsl_odeiv2_step * step = gsl_odeiv2_step_alloc(T, 8);
    gsl_odeiv2_control * control = gsl_odeiv2_control_yp_new(1e-14, 1e-14);
    gsl_odeiv2_evolve * evol = gsl_odeiv2_evolve_alloc(8);
    gsl_params gsl_parameters = {commondata, params, metric};
    gsl_odeiv2_system sys = {ode_gsl_wrapper_massive, NULL, 8, &gsl_parameters};

    double y_c[8];
    double t = 0.0, dt = 0.01; // t is proper time τ
    for (int j = 0; j < 8; j++) { y_c[j] = start_y[j]; }

    // Setup output file
    FILE *fp = fopen("massive_particle_path.txt", "w");
    if (fp == NULL) { 
        fprintf(stderr, "Error: Could not open massive_particle_path.txt for writing.\n");
        exit(1); 
    }
    fprintf(fp, "# ProperTime_tau\tCoordTime_t\tx\ty\tz\tu^t\tu^x\tu^y\tu^z\n");

    printf("Starting debug trace for single massive particle...\n");
    printf("Step | Proper Time (τ) | Coord Time (t) |      x     |      y     |      z     |      u^t   \n");
    printf("-------------------------------------------------------------------------------------------\n");

    // Main Integration Loop
    for (int i = 0; i < 2000000; i++) {
        int status = gsl_odeiv2_evolve_apply(evol, control, step, &sys, &t, 1e10, &dt, y_c);
        
        // Write full state to file
        fprintf(fp, "%.6e\t%.6e\t%.6e\t%.6e\t%.6e\t%.6e\t%.6e\t%.6e\t%.6e\n", 
                t, y_c[0], y_c[1], y_c[2], y_c[3], y_c[4], y_c[5], y_c[6], y_c[7]);

        if (i % 500 == 0) {
            printf("%4d | %15.4e | %14.4f | %10.4f | %10.4f | %10.4f | %10.4f\n",
                   i, t, y_c[0], y_c[1], y_c[2], y_c[3], y_c[4]);
        }

        const double r_sq = y_c[1]*y_c[1] + y_c[2]*y_c[2] + y_c[3]*y_c[3];
        // Event horizon radius for a Kerr black hole
        const double r_horizon = commondata->M_scale * (1.0 + sqrt(1.0 - commondata->a_spin*commondata->a_spin));

        // Termination Conditions
        if (status != GSL_SUCCESS) { printf("Termination: GSL ERROR (status = %d)\n", status); break; }
        if (r_sq < r_horizon*r_horizon) { printf("Termination: Fell below event horizon (r=%.2f)\n", sqrt(r_sq)); break; }
        if (r_sq > r_escape*r_escape) { printf("Termination: Escaped to r > %.1f\n", r_escape); break; }
        if (fabs(y_c[4]) > ut_max) { printf("Termination: Runaway u^t > %.1e\n", ut_max); break; }
        if (fabs(y_c[0]) > t_max_integration) { printf("Termination: Exceeded max integration time t > %.1f\n", t_max_integration); break; }
    }

    // Copy final state to output and clean up
    for(int j=0; j<8; j++) { final_y_state[j] = y_c[j]; }
    fclose(fp);
    gsl_odeiv2_evolve_free(evol);
    gsl_odeiv2_control_free(control);
    gsl_odeiv2_step_free(step);
    """
    
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        cfunc_type=cfunc_type,
        name=name,
        params=params,
        body=body, 
        include_CodeParameters_h=True
    )

### 6.C.3: The Main Simulation Loop Orchestrator

This function generates the top-level orchestrator for a full production run. It is responsible for setting up the particle ensemble, looping through time, saving snapshots, and calling the single-particle integrator for each particle at each time step.

**How the C Function Works:**
The generated C function `run_mass_integrator_production()` performs the following sequence of operations:
1.  **Initial Conditions Dispatch**: It reads the `initial_conditions_type` parameter from the `commondata` struct and calls the appropriate generator function (`generate_disk_initial_conditions`, `generate_spiral_galaxy_initial_conditions`, etc.) to create the initial state for all particles.
2.  **Data Structure Setup**: It populates the main array of `mass_particle_state_t` structs, which is the format used for saving snapshots.
3.  **Main Time Loop**: It enters a `for` loop that iterates from `t=0` to `t_final` in steps of `snapshot_every_t`.
4.  **Save Snapshot**: At the beginning of each iteration, it saves the current state of all active particles to a binary snapshot file (`mass_blueprint_t_xxxx.bin`).
5.  **Parallel Integration**: It then uses an OpenMP `#pragma omp parallel for` loop to distribute the work of integrating all particles to the next snapshot time. Inside this loop, each thread calls `integrate_single_particle()` for a subset of the particles.
6.  **Update State**: After the integration step, it updates the main particle state array with the new positions and velocities, marking any terminated particles with `NAN` so they are excluded from future steps and snapshots.

In [ ]:
def run_mass_integrator_production():
    """
    Generates the C orchestrator for the production run.
    
    This definitive version acts as a dispatcher. It reads the 
    initial_conditions_type parameter and calls the appropriate particle 
    generator function before starting the full integration and snapshotting loop.
    """
    print(" -> Generating C orchestrator: run_mass_integrator_production() [Dispatcher Version]...")
    
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "<math.h>", "<sys/stat.h>", "<string.h>", "<stdio.h>", "<stdlib.h>"]
    desc = r"""@brief Orchestrates the full production run for the mass integrator."""
    name = "run_mass_integrator_production"
    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const metric_params *restrict metric"
    
    body = r"""
    // Step 1: Allocate memory for the flat initial state array
    const int max_particles = commondata->disk_num_r * commondata->disk_num_phi;
    double *y_initial_states = (double *)malloc(sizeof(double) * 8 * max_particles);
    if (y_initial_states == NULL) { fprintf(stderr, "Error: Failed to allocate memory for initial states.\n"); exit(1); }
    int num_particles = 0;

    // --- Step 2: Dispatch to the correct initial conditions generator ---
    printf("Generating initial conditions of type: %s\n", commondata->initial_conditions_type);
     if (strcmp(commondata->initial_conditions_type, "SpiralGalaxy") == 0) {
        num_particles = generate_spiral_galaxy_initial_conditions(commondata, params, y_initial_states);
    } else if (strcmp(commondata->initial_conditions_type, "BarredFlocculentSpiral") == 0) {
        num_particles = generate_barred_flocculent_spiral_ic(commondata, params, y_initial_states);
    } else {// Default to KeplerianDisk
        if (strcmp(commondata->initial_conditions_type, "KeplerianDisk") != 0) {
            printf("Warning: Unrecognized initial_conditions_type '%s'. Defaulting to KeplerianDisk.\n", commondata->initial_conditions_type);
        }
        num_particles = generate_disk_initial_conditions(commondata, params, y_initial_states);
    }
    printf("Generated %d particles.\n", num_particles);

    // Step 3: Create and populate our primary data structure, an array of mass_particle_state_t
    mass_particle_state_t *particle_states = (mass_particle_state_t *)malloc(sizeof(mass_particle_state_t) * num_particles);
    if (particle_states == NULL) { fprintf(stderr, "Error: Failed to allocate memory for particle states.\n"); exit(1); }
    for (int i=0; i<num_particles; i++) {
        double *y_start = &y_initial_states[i*8];
        particle_states[i].id = i;
        particle_states[i].pos[0] = y_start[1];
        particle_states[i].pos[1] = y_start[2];
        particle_states[i].pos[2] = y_start[3];
        particle_states[i].u[0] = y_start[4];
        particle_states[i].u[1] = y_start[5];
        particle_states[i].u[2] = y_start[6];
        particle_states[i].u[3] = y_start[7];
        
        const double r = sqrt(y_start[1]*y_start[1] + y_start[2]*y_start[2]);
        particle_states[i].lambda_rest = commondata->disk_lambda_rest_at_r_min * pow(r / commondata->disk_r_min, 0.75);
        particle_states[i].j_intrinsic = (float)pow(r / commondata->disk_r_min, -3.0);
    }
    free(y_initial_states); 

    // Step 4: Create the output directory
    mkdir(commondata->output_folder, 0755);

    // Step 5: Main Time Evolution and Snapshotting Loop
    int snapshot_count = 0;
    for (double current_t = 0; current_t <= commondata->t_final; current_t += commondata->snapshot_every_t) {
        char filename[200];
        snprintf(filename, 200, "%s/mass_blueprint_t_%04d.bin", commondata->output_folder, snapshot_count);
        printf("Saving snapshot: %s (t=%.2f)\n", filename, current_t);
        
        FILE *fp_out = fopen(filename, "wb");
        if (fp_out == NULL) { exit(1); }
        
        int active_particles = 0;
        for(int i=0; i<num_particles; i++) {
            if (!isnan(particle_states[i].pos[0])) active_particles++;
        }
        fwrite(&active_particles, sizeof(int), 1, fp_out);
        for (int i=0; i<num_particles; i++) {
            if (!isnan(particle_states[i].pos[0])) {
                fwrite(&particle_states[i], sizeof(mass_particle_state_t), 1, fp_out);
            }
        }
        fclose(fp_out);
        snapshot_count++;

        if (current_t >= commondata->t_final) break;

        // Evolve all particles for one snapshot interval
        const double t_next_snapshot = current_t + commondata->snapshot_every_t;
        #pragma omp parallel for
        for (int i = 0; i < num_particles; i++) {
            if (isnan(particle_states[i].pos[0])) continue;

            double y_particle[8];
            y_particle[0] = current_t;
            y_particle[1] = particle_states[i].pos[0];
            y_particle[2] = particle_states[i].pos[1];
            y_particle[3] = particle_states[i].pos[2];
            y_particle[4] = particle_states[i].u[0];
            y_particle[5] = particle_states[i].u[1];
            y_particle[6] = particle_states[i].u[2];
            y_particle[7] = particle_states[i].u[3];

            int status = integrate_single_particle(commondata, params, metric, y_particle[0], t_next_snapshot, y_particle);
            
            if (status != 0) {
                particle_states[i].pos[0] = NAN; // Mark particle as terminated
            } else {
                particle_states[i].pos[0] = y_particle[1];
                particle_states[i].pos[1] = y_particle[2];
                particle_states[i].pos[2] = y_particle[3];
                particle_states[i].u[0] = y_particle[4];
                particle_states[i].u[1] = y_particle[5];
                particle_states[i].u[2] = y_particle[6];
                particle_states[i].u[3] = y_particle[7];
            }
        }
    }
    free(particle_states);
    """
    cfc.register_CFunction(includes=includes, desc=desc, name=name, params=params, body=body)

### 6.C.4: The `main()` C Function and Entry Point

This is the final C-generating function. It creates the `main()` function, which is the entry point for the entire compiled C program. It acts as the master orchestrator, parsing command-line arguments and dispatching to either the single-particle debug mode or the full production run.

**How the C Function Works:**
The generated `main()` function performs these steps in order:
1.  **Initialize Parameters**: It calls `commondata_struct_set_to_default()` to set all parameters to their default values, and then calls `cmdline_input_and_parfile_parser()` to overwrite these defaults with values from the `.par` file and any command-line arguments.
2.  **Set Metric Type**: It determines whether to use the `Kerr` or `Schwarzschild` metric type based on whether the spin parameter `a_spin` is zero.
3.  **Dispatch to Run Mode**: It checks the boolean parameter `run_in_debug_mode`.
    *   If `true`, it executes the debug workflow: it reads initial conditions from a text file (`particle_debug_initial_conditions.txt`), calls `set_initial_conditions_massive` to compute the full state vector, optionally calls `check_conservation_massive` to check initial conserved quantities, calls `integrate_single_particle_DEBUG` to trace the path, and finally calls `check_conservation_massive` again to report the final error.
    *   If `false`, it prints a summary of the simulation parameters and calls the `run_mass_integrator_production()` orchestrator to start the full N-body simulation.

In [ ]:
def main():
    """
    Generates the main() C function.
    
    This final version restores the detailed parameter printout for production runs.
    The production run logic is now handled by the run_mass_integrator_production dispatcher.
    """
    print(" -> Generating C entry point: main() [Final Version]...")
    
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "<string.h>", "<stdio.h>", "<stdlib.h>"]
    
    desc = r"""@brief Main entry point for the massive particle geodesic integrator."""
    cfunc_type = "int"
    name = "main"
    params = "int argc, const char *argv[]"

    body = r"""
    // Step 1: Initialize structs and parameters
    commondata_struct commondata;
    params_struct params; // This struct is currently unused but required by function signatures.
    metric_params metric;
    
    commondata_struct_set_to_default(&commondata);
    cmdline_input_and_parfile_parser(&commondata, argc, argv);
    
    metric.type = (commondata.a_spin == 0.0) ? Schwarzschild : Kerr;

    // Step 2: Check the run mode and execute the appropriate logic
    if (commondata.run_in_debug_mode) {
        /***********************************/
        /*** SINGLE-PARTICLE DEBUG MODE ***/
        /***********************************/
        particle_initial_state_t initial_state;
        initial_state.id = 0;

        const char *filename = "particle_debug_initial_conditions.txt";
        FILE *fp_in = fopen(filename, "r");
        
        // --- RESTORED FALLBACK LOGIC ---
        // If the file doesn't exist, create it with default values.
        if (fp_in == NULL) {
            printf("File '%s' not found. Creating it with default values.\n", filename);
            fp_in = fopen(filename, "w");
            if (fp_in == NULL) { 
                fprintf(stderr, "Error: Could not create '%s'.\n", filename); 
                return 1; 
            }
            fprintf(fp_in, "# Format: t_initial pos_x pos_y pos_z u_x u_y u_z\n");
            fprintf(fp_in, "# u_i are the spatial components of the 4-velocity (dx/dτ).\n");
            fprintf(fp_in, "0.0 10.0  0.0  0.0   0.0  0.363232  0.0\n");
            fclose(fp_in);
            
            // Re-open the newly created file for reading
            fp_in = fopen(filename, "r");
            if (fp_in == NULL) { 
                fprintf(stderr, "Error: Could not re-open '%s' for reading.\n", filename); 
                return 1; 
            }
        }
        // --- END OF RESTORED LOGIC ---

        char line[256];
        while (fgets(line, sizeof(line), fp_in)) {
            if (line[0] != '#') {
                sscanf(line, "%lf %lf %lf %lf %lf %lf %lf", 
                       &initial_state.pos[0], &initial_state.pos[1], &initial_state.pos[2], &initial_state.pos[3],
                       &initial_state.u_spatial[0], &initial_state.u_spatial[1], &initial_state.u_spatial[2]);
                break; 
            }
        }
        fclose(fp_in);

        printf("--- Single Particle Debug Run ---\n");
        printf("  pos = (t=%.4f, x=%.4f, y=%.4f, z=%.4f)\n", initial_state.pos[0], initial_state.pos[1], initial_state.pos[2], initial_state.pos[3]);
        printf("  u_spatial = (%.4f, %.4f, %.4f)\n", initial_state.u_spatial[0], initial_state.u_spatial[1], initial_state.u_spatial[2]);

        double y_start[8], y_final[8];
        

        set_initial_conditions_massive(&initial_state, &commondata, y_start);


        printf("\nInitial State Vector (y_start):\n");
        printf("  t=%.2f, x=%.2f, y=%.2f, z=%.2f\n", y_start[0], y_start[1], y_start[2], y_start[3]);
        printf("  u^t=%.4f, u^x=%.4f, u^y=%.4f, u^z=%.4f\n\n", y_start[4], y_start[5], y_start[6], y_start[7]);

        if(commondata.perform_conservation_check) {
            double E_i, Lx_i, Ly_i, Lz_i, Q_i, E_f, Lx_f, Ly_f, Lz_f, Q_f;
            check_conservation_massive(&commondata, &params, &metric, y_start, &E_i, &Lx_i, &Ly_i, &Lz_i, &Q_i);
            integrate_single_particle_DEBUG(&commondata, &params, &metric, y_start, y_final);
            check_conservation_massive(&commondata, &params, &metric, y_final, &E_f, &Lx_f, &Ly_f, &Lz_f, &Q_f);
        } else {
            integrate_single_particle_DEBUG(&commondata, &params, &metric, y_start, y_final);
        }
        
        printf("\nDebug run finished. Trajectory saved to 'massive_particle_path.txt'.\n");

    } else {
        /***********************************/
        /*** FULL DISK PRODUCTION MODE ***/
        /***********************************/
        printf("----------------------------------------\n");
        printf("Massive Particle Integrator\n");
        printf("----------------------------------------\n");
        printf("Metric Settings:\n");
        printf("  Metric Type             = %s (a=%.2f, M=%.2f)\n", (metric.type == Kerr) ? "Kerr" : "Schwarzschild", commondata.a_spin, commondata.M_scale);
        printf("\nIntegration Settings:\n");
        printf("  Max Integration Time    = %.1f M\n", commondata.t_final);
        printf("  Snapshot Every          = %.1f M\n", commondata.snapshot_every_t);
        printf("  Escape Radius           = %.1f M\n", commondata.r_escape);
        printf("\nInitial Conditions:\n");
        printf("  Generator Type          = %s\n", commondata.initial_conditions_type);
        printf("  Num Particles (r x phi) = %d x %d\n", commondata.disk_num_r, commondata.disk_num_phi);
        printf("  Disk Radial Min/Max     = %.2f / %.2f M\n", commondata.disk_r_min, commondata.disk_r_max);
        printf("----------------------------------------\n\n");

        run_mass_integrator_production(&commondata, &params, &metric);
        printf("\nProduction run finished successfully.\n");
    }
    
    return 0;
    """
    
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        cfunc_type=cfunc_type,
        name=name,
        params=params,
        body=body
    )

### Temporary Validation `main()` Function

This function, `main_ic_validation`, generates a **temporary, alternative** `main()` function. Its sole purpose is to provide a simple, self-contained program to test the C function `calculate_ut_uphi_from_r` in isolation. This is a powerful debugging and validation technique.

**Why it's Needed:**
The `calculate_ut_uphi_from_r` function implements a complex, numerically sensitive formula. Before integrating it into the full simulation, we need to be confident that it produces correct and physically reasonable results across a wide range of inputs. Running the full simulation for each test case would be slow and cumbersome. This validation `main()` allows for rapid, targeted testing of this single critical component.

**How the C Function Works:**
The generated `main()` function does not run a simulation. Instead, it:
1.  Initializes a `commondata` struct to access default parameters like `M_scale`.
2.  Defines a hardcoded C array of test cases. Each test case is a pair of `{a_spin, r_initial}` values, specifically chosen to probe challenging scenarios (e.g., high spin, near the ISCO, retrograde orbits).
3.  Loops through this array of test cases.
4.  In each iteration, it updates the `commondata.a_spin` value and calls `calculate_ut_uphi_from_r` with the test `r_initial`.
5.  It then uses `printf` to print the inputs (`a_spin`, `r_initial`) and the outputs (`u^t`, `u^\phi`) to the console in a neatly formatted table.

By compiling and running the project with this `main()` function, a developer can immediately inspect the output table and verify that the initial condition function is behaving correctly (e.g., not producing `NaN` or `inf` values) for all scenarios before proceeding with the full integration.


In [ ]:
def main_ic_validation():
    """
    Generates a TEMPORARY main() C function for the sole purpose of validating
    the output of the new, numerically stable 'calculate_ut_uphi_from_r' function.
    
    This main() will loop through a pre-defined set of (a, r) pairs, call the
    C function, and print the resulting u^t and u^phi to the console.
    """
    print(" -> Registering TEMPORARY main() function for IC validation...")
    
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "<stdio.h>"]
    desc = r"""@brief Temporary main() for validating initial conditions.
    
    This program tests the numerically stable 'calculate_ut_uphi_from_r' C function
    by calling it for a range of black hole spins 'a' and orbital radii 'r'.
    It prints the resulting 4-velocity components u^t and u^phi to the console."""
    cfunc_type = "int"
    name = "main"
    params = "int argc, const char *argv[]"

    body = r"""
    // Suppress unused parameter warnings for this temporary main
    (void)argc;
    (void)argv;

    // Initialize the commondata struct to get default M_scale
    commondata_struct commondata;
    params_struct params; // Dummy struct, not used but required by function signature
    commondata_struct_set_to_default(&commondata);

    printf("--- Initial Condition Validation for calculate_ut_uphi_from_r() ---\n");
    printf("      (Using numerically stable method)\n\n");
    printf("%-10s | %-10s | %-20s | %-20s\n", "a_spin", "r_initial", "u^t (dt/d_tau)", "u^phi (d_phi/d_tau)");
    printf("--------------------------------------------------------------------------\n");

    // Define the array of test cases: {a_spin, r_initial}
    const double test_cases[][2] = {
        // Prograde, high spin (a=0.99), near ISCO (~1.23M)
        {0.99, 1.5},
        {0.99, 2.0},
        {0.99, 6.0},
        
        // Schwarzschild (a=0.0), near ISCO (6M)
        {0.0, 6.0},
        {0.0, 8.0},
        {0.0, 20.0},

        // Retrograde, high spin (a=-0.99), near ISCO (~8.98M)
        {-0.99, 9.0},
        {-0.99, 12.0},
        {-0.99, 50.0}
    };
    const int num_test_cases = sizeof(test_cases) / sizeof(test_cases[0]);

    for (int i = 0; i < num_test_cases; ++i) {
        commondata.a_spin = test_cases[i][0];
        const double r_initial = test_cases[i][1];
        
        double ut, uphi;
        
        // Call the C function we want to test
        calculate_ut_uphi_from_r(r_initial, &commondata, &params, &ut, &uphi);
        
        printf("%-10.2f | %-10.2f | %-20.10f | %-20.10f\n", 
               commondata.a_spin, r_initial, ut, uphi);
    }

    printf("--------------------------------------------------------------------------\n");
    printf("Validation complete. Check for NaN/inf values and physical consistency.\n");

    return 0;
    """
    
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        cfunc_type=cfunc_type,
        name=name,
        params=params,
        body=body
    )

<a id='assemble_project'></a>
# Step 7: Project Assembly and Compilation

This is the final phase of the notebook. The functions and cells below bring all the previously defined pieces together to construct the complete, compilable C project.

<a id='register_structs'></a>
### 7.a: Registering Core C Data Structures

This function generates the C `typedef`s for all the custom data structures (`struct`s and `enum`s) used by the project and registers this block of C code to be written into the master header file, `BHaH_defines.h`.

**Generated Structures:**
*   `metric_struct`: Contains the 10 unique components of the symmetric 4x4 metric tensor $g_{\mu\nu}$.
*   `connection_struct`: Contains the 40 unique components of the Christoffel symbols $\Gamma^\alpha_{\mu\nu}$ (symmetric in $\mu, \nu$).
*   `Metric_t`: An `enum` to provide human-readable names for the different spacetime choices (`Schwarzschild`, `Kerr`, etc.).
*   `metric_params`: A small struct to hold the `Metric_t` choice.
*   `gsl_params`: A "carrier" struct used to pass all necessary simulation data (pointers to `commondata`, `params`, and `metric` structs) through GSL's generic `void*` parameter system.
*   `particle_initial_state_t`: A struct for reading initial conditions for a single particle in debug mode.
*   `mass_particle_state_t`: The definitive struct that defines the binary format for the production-run snapshot files. The `__attribute__((packed))` directive is a compiler-specific extension (supported by GCC and Clang) that tells the compiler not to insert any padding bytes between the members of the struct. This is crucial for writing binary files, as it ensures that the memory layout of the struct in the C program exactly matches the byte-for-byte layout that will be written to disk, guaranteeing file portability and readability across different systems.


In [ ]:
def register_custom_structures_and_params():
    """
    Generates C code for all custom structs and enums, then registers them with BHaH.
    """
    print("Registering custom C data structures for mass integrator...")
    
    metric_components = [f"g{nu}{mu}" for nu in range(4) for mu in range(nu, 4)]
    metric_struct_str = "typedef struct { double " + "; double ".join(metric_components) + "; } metric_struct;"
    
    connection_components = [f"Gamma4UDD{i}{j}{k}" for i in range(4) for j in range(4) for k in range(j, 4)]
    connections_struct_str = "typedef struct { double " + "; double ".join(connection_components) + "; } connection_struct;"

    other_structs = r"""
typedef enum { Schwarzschild, Kerr, Numerical, Schwarzschild_Standard } Metric_t;
typedef struct { Metric_t type; } metric_params;

typedef struct { 
    const commondata_struct *commondata; 
    const params_struct *params; 
    const metric_params *metric; 
} gsl_params;

// Struct for reading initial conditions from a debug file (unchanged)
typedef struct {
    int id;
    double pos[4];
    double u_spatial[3];
} particle_initial_state_t;

// DEFINITIVE struct for a single particle in the output snapshot files.
// This is the format that will be written to disk.
typedef struct {
    int id;
    double pos[3];          // x, y, z position
    double u[4];    //u^t u^x, u^y, u^z 
    double lambda_rest;     // Rest-frame emission wavelength (nm)
    float j_intrinsic;      // Rest-frame intrinsic emissivity (intensity)
} __attribute__((packed)) mass_particle_state_t;
"""

    Bdefines_h.register_BHaH_defines("data_structures", f"{metric_struct_str}\n{connections_struct_str}\n{other_structs}")
    print(" -> Registered all necessary data structures, including mass_particle_state_t.")

<a id='final_build'></a>
### 7.b: Final Build Command

This is the main execution block of the notebook. It orchestrates the entire `nrpy` build process from start to finish.

1.  **Register All Components**: It calls all the C-generating Python functions that we have defined throughout the notebook (e.g., `g4DD_kerr_schild`, `calculate_ode_rhs_massive`, `main`). This populates `nrpy`'s internal library (`cfc.CFunction_dict`) with the complete definitions for all our custom C functions.
2.  **Generate Parameter Handling Files**: It calls the necessary functions from the BHaH infrastructure:
    *   `CPs.write_CodeParameters_h_files()`: Generates `set_CodeParameters.h`, which allows C functions to access runtime parameters as local constants.
    *   `CPs.register_CFunctions_params_commondata_struct_set_to_default()`: Generates C functions to initialize parameters to their default values.
    *   `cmdline_input_and_parfiles.generate_default_parfile()`: Creates the `mass_integrator.par` file with all registered parameters and their defaults.
    *   `cmdline_input_and_parfiles.register_CFunction_cmdline_input_and_parfile_parser()`: Generates the C function that reads the `.par` file and command-line arguments.
3.  **Generate Headers and Makefile**: It calls the final build functions:
    *   `Bdefines_h.output_BHaH_defines_h()`: Writes `BHaH_defines.h`, containing all `typedef`s, `struct`s, and `#define` macros.
    *   `Makefile.output_CFunctions_function_prototypes_and_construct_Makefile()`: Iterates through the populated `CFunction_dict` to write all the `.c` source files, the `BHaH_function_prototypes.h` header, and the `Makefile` itself. The `addl_CFLAGS` and `addl_libraries` arguments are used to add the necessary compiler and linker flags to use the external GSL library and enable OpenMP for parallelization.
    *   The `gsl-config` command is a utility provided by the GSL library that automatically outputs the correct compiler flags (`--cflags`) and linker flags (`--libs`) needed to compile and link against it. Using this utility makes the build process portable across different systems where GSL might be installed in different locations.
    *   The `-fopenmp` flag enables support for OpenMP, which is used to parallelize the main integration loop in `run_mass_integrator_production`.

After this cell is run, a complete, self-contained, and ready-to-compile C project will exist in the `project/mass_integrator/` directory.

In [ ]:
# In V1_1_mass_geodesic.ipynb, Cell ID a0eb212d (Final Build Script)
print("\nAssembling and building C project for the massive particle integrator...")
os.makedirs(project_dir, exist_ok=True)

# --- Step 1: Register all C-generating functions in the correct order ---
print(" -> Registering C data structures and functions...")
register_custom_structures_and_params()

# Register symbolic recipes and C-generating functions for physics
# symbolic_ut_uphi_from_r() is called by the next function
calculate_ut_uphi_from_r() # The new, required helper function

# Register C workers for metrics and connections
g4DD_kerr_schild(); con_kerr_schild()
g4DD_schwarzschild_cartesian(); con_schwarzschild_cartesian()

# Register C dispatchers
g4DD_metric(); connections()

# Register C engines for the core logic
calculate_ode_rhs_massive()
ode_gsl_wrapper_massive()
set_initial_conditions_massive()
check_conservation_massive()
integrate_single_particle() 
integrate_single_particle_DEBUG()
run_mass_integrator_production()

# Register the production-run orchestrators
generate_disk_initial_conditions()
generate_spiral_galaxy_initial_conditions()
generate_barred_flocculent_spiral_ic()
main()
#main_ic_validation()


# --- Step 2: Call BHaH infrastructure functions to generate the build system ---
print(" -> Generating BHaH infrastructure files...")
# Generate set_CodeParameters.h and its variants
CPs.write_CodeParameters_h_files(project_dir=project_dir)
# Register C functions to set parameters to default values
CPs.register_CFunctions_params_commondata_struct_set_to_default()
# Generate the default parameter file (mass_integrator.par)
cmdline_input_and_parfiles.generate_default_parfile(project_dir=project_dir, project_name=project_name)

# Register the C function that parses the command line and parameter file
cmdline_input_and_parfiles.register_CFunction_cmdline_input_and_parfile_parser(
    project_name=project_name,
    cmdline_inputs=[
        'M_scale', 'a_spin', 't_max_integration', 'flatness_threshold', 
        'r_escape', 'ut_max', 'perform_conservation_check', 'run_in_debug_mode',
        'initial_conditions_type',
        'spiral_galaxy_num_arms',  
        'spiral_galaxy_arm_tightness',
        'bar_length', 'bar_aspect_ratio', 'bulge_radius',
        'arm_particle_density', 'arm_clumpiness_factor', 'arm_clump_size',
        'bar_density_factor', 'bulge_density_factor'
]
)

# --- Step 3: Generate the final C code, headers, and Makefile ---
print("\nGenerating BHaH master header file (BHaH_defines.h)...")
Bdefines_h.output_BHaH_defines_h(project_dir=project_dir)

# Note: SIMD intrinsics are not used in this project, but the helper is harmless.
print("Copying required helper files...")
gh.copy_files(
    package="nrpy.helpers",
    filenames_list=["simd_intrinsics.h"],
    project_dir=project_dir,
    subdirectory="simd",
)

print("Generating all C source files, function prototypes, and the Makefile...")
# Add required GSL and OpenMP flags to the compiler
addl_CFLAGS = ["-Wall -Wextra -g $(shell gsl-config --cflags) -fopenmp"]
addl_libraries = ["$(shell gsl-config --libs) -fopenmp"]

Makefile.output_CFunctions_function_prototypes_and_construct_Makefile(
    project_dir=project_dir,
    project_name=project_name,
    exec_or_library_name="mass_integrator", # The name of our final executable
    addl_CFLAGS=addl_CFLAGS,
    addl_libraries=addl_libraries,
)

print(f"\nFinished! A C project has been generated in '{project_dir}/'")
print(f"To build, navigate to this directory in your terminal and type 'make'.")
print(f"To run, type './mass_integrator'.")